In [1]:
import os
import pandas as pd
import pycipher as pc
import re
from collections import namedtuple

from itertools import combinations
from time import sleep

pd.set_option('display.max_columns', None)
os.chdir('/home/mproffitt/src/pykryptos')
from pykryptos.cipher import Decipher

from enigma.machine import EnigmaMachine

## Set up standard arguments
The following block overrides the arguments presented by ArgumentParser when passed to the Decipher class

## Standard settings

In [2]:
fulltext1="""EMUFPHZLRFAXYUSDJKZLDKRNSHGNFIVJ
YQTQUXQBQVYUVLLTREVJYQTMKYRDMFD
VFPJUDEEHZWETZYVGWHKKQETGFQJNCE
GGWHKK?DQMCPFQZDQMMIAGPFXHQRLG
TIMVMZJANQLVKQEDAGDVFRPJUNGEUNA
QZGZLECGYUXUEENJTBJLBQCRTBJDFHRR
YIZETKZEMVDUFKSJHKFWHKUWQLSZFTI
HHDDDUVH?DWKBFUFPWNTDFIYCUQZERE
EVLDKFEZMOQQJLTTUGSYQPFEUNLAVIDX
FLGGTEZ?FKZBSFDQVGOGIPUFXHHDRKF
FHQNTGPUAECNUVPDJMQCLQUMUNEDFQ
ELZZVRRGKFFVOEEXBDMVPNFQXEZLGRE
DNQFMPNZGLFLPMRJQYALMGNUVPDXVKP
DQUMEBEDMHDAFMJGZNUPLGEWJLLAETG
""".replace('\n','')

fulltext2 = """ENDYAHROHNLSRHEOCPTEOIBIDYSHNAIA
CHTNREYULDSLLSLLNOHSNOSMRWXMNE
TPRNGATIHNRARPESLNNELEBLPIIACAE
WMTWNDITEENRAHCTENEUDRETNHAEOE
TFOLSEDTIWENHAEIOYTEYQHEENCTAYCR
EIFTBRSPAMHHEWENATAMATEGYEERLB
TEEFOASFIOTUETUAEOTOARMAEERTNRTI
BSEDDNIAAHTTMSTEWPIEROAGRIEWFEB
AECTDDHILCEIHSITEGOEAOSDDRYDLORIT
RKLMLEHAGTDHARDPNEOHMGFMFEUHE
ECDMRIPFEIMEHNLSSTTRTVDOHW?OBKR
UOXOGHULBSOLIFBBWFLRVQQPRNGKSSO
TWTQSJQSSEKZZWATJKLUDIAWINFBNYP
VTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
""".replace('\n', '')

In [3]:
arguments = {
    'time':       'now',
    'increment':  'second',
    'speed':      0.000001,
    'alphabet':   ''.join(Decipher.clock_alphabet),
    'keyword':    '',
    'method':     'add',
    'ciphertext': Decipher.ciphertext,
    'kryptos':    '',
    'preop':      None
}
arguments = namedtuple('Cipher', arguments.keys())(**arguments)

## Decipher object
Many of the techniques we're trying in this notebook rely on the decipher class

In [4]:
decipher = Decipher(arguments)

## Cipher and Vigenere classes

The base class `Cipher` from pycipher does not currently allow appending a keyword. It also has hard coded alphabets in the a2i and i2a functions.

This causes problems with the Vigenere class as it becomes less than complete.

We redefine the base class here

In [5]:
class Cipher(object):
    _alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    _keyword  = ''
    
    def encipher(self,string):
        return string
        
    def decipher(self,string):
        return string
        
    def a2i(self,ch):
        return list(self.alphabet).index(ch.upper())

    def i2a(self,i):
        return list(self.alphabet)[(i % 26)]
    
    @property
    def keyword(self):
        return self._keyword
    
    @keyword.setter
    def keyword(self, keyword):
        self._keyword = self.remove_punctuation(keyword)
    
    @property
    def alphabet(self):
        return '{}{}'.format(
            self._keyword,
            ''.join(
                sorted(
                    list(set(list(self._alphabet)) - set(list(self._keyword)))
                )
            )
        )
    
    def remove_punctuation(self,text,filter='[^A-Z]'):
        return re.sub(filter,'',text.upper())
    


Re-definition of the Vigenere class so it inherits the local Cipher class rather than pycipher.Cipher

In [6]:
class Vigenere(Cipher):
    """The Vigenere Cipher has a key consisting of a word e.g. 'FORTIFICATION'.
    This cipher encrypts a letter according to the Vigenere tableau, the algorithm can be 
    seen e.g. http://practicalcryptography.com/ciphers/vigenere-gronsfeld-and-autokey-cipher/
    
    :param key: The keyword, any word or phrase will do. Must consist of alphabetical
                characters only, no punctuation of numbers.        
    """
    def __init__(self, key='fortification'):
        self.key = [k.upper() for k in key]
        
    def encipher(self,string):
        """Encipher string using Vigenere cipher according to initialised key. Punctuation and whitespace
        are removed from the input.       

        Example::

            ciphertext = Vigenere('HELLO').encipher(plaintext)     

        :param string: The string to encipher.
        :returns: The enciphered string.
        """           
        string = self.remove_punctuation(string)
        ret = ''
        for (i,c) in enumerate(string):
            i = i%len(self.key)
            ret += self.i2a(self.a2i(c) + self.a2i(self.key[i]))
        return ret    

    def decipher(self,string):
        """Decipher string using Vigenere cipher according to initialised key. Punctuation and whitespace
        are removed from the input.       

        Example::

            plaintext = Vigenere('HELLO').decipher(ciphertext)     

        :param string: The string to decipher.
        :returns: The enciphered string.
        """               
        string = self.remove_punctuation(string)
        ret = ''
        for (i,c) in enumerate(string):
            i = i%len(self.key)
            ret += self.i2a(self.a2i(c) - self.a2i(self.key[i]))
        return ret    

## K0 (Morse)

## K1

In [7]:
vig1 = Vigenere('PALIMPSEST')
vig1.keyword = 'KRYPTOS'
k1 = vig1.decipher(fulltext1[:63])
k1

'BETWEENSUBTLESHADINGANDTHEABSENCEOFLIGHTLIESTHENUANCEOFIQLUSION'

## K2

In [8]:
vig2 = Vigenere(key='BSCISSAA')
vig2.keyword = 'KRYPTOS'
k2 = vig2.decipher(fulltext1)[63:]
k2

'ITWASTOTALLYINVISIBLEHOWSTHATPOSSIBLETHEYUSEDTHEEARTHSMAGNETICFIELDXTHEINFORMATIONWASGATHEREDANDTRANSMITTEDUNDERGRUUNDTOANUNKNOWNLOCATIONXDOESLANGLEYKNOWABOUTTHISTHEYSHOULDITSBURIEDOUTTHERESOMEWHEREXWHOKNOWSTHEEXACTLOCATIONONLYWWTHISWASHISLASTMESSAGEXTHIRTYEIGHTDEGREESFIFTYSEVENMINUTESSIXPOINTFIVESECONDSNORTHSEVENTYSEVENDEGREESEIGHTMINUTESFORTYFOURSECONDSWESTIDBYROWS'

## K3
There is no standard cipher which works for K3 within the pycipher or pykryptos libraries.

There are 3 functions necessary for this to take place so we define them here.

* Function 1: `chunkstring` Split a string into n-sized chunks and return as a list
* Function 2: `rotate` Take a list of chunks and rotate it 90°
* Function 3: `transpose` The actual decipher for K3.

The Transpose function first writes the message out as 24 columns, then rotates it by 90°, then splits it to 8 columns and rotates it 90°. Finally it splits it to len(2) cols and rotate 90° which gives the final message.

I couldn't find an example out there of how to achieve this in code so I wrote my own method to do it.

In [9]:
def chunkstring(string, length):
    return list((string[0+i:length+i] for i in range(0, len(string), length)))

def rotate(chunks, direction='ltr'):
    length =  len(''.join(chunks))
    new_message = ''
    while True:
        for index, element in enumerate(chunks):
            if len(element) == 0:
                new_message += ''
                continue
            if direction == 'ltr':
                new_message += element[0]
                chunks[index] = element[1:]
            elif direction == 'rtl':
                new_message += element[-1]
                chunks[index] = element[:-1]
        
        if len(new_message) == length:
            break
    return new_message

def transpose(message, initial_cols=24, secondary_cols=8):
    message = chunkstring(message, initial_cols) # 24 initial columns
    message = chunkstring(rotate(message[::-1]), secondary_cols)
    message = chunkstring(rotate(message[::-1]), len(message))
    return ''.join(message)

In [10]:
k3 = transpose(fulltext2[:-98])
k3

'SLOWLYDESPARATLYSLOWLYTHEREMAINSOFPASSAGEDEBRISTHATENCUMBEREDTHELOWERPARTOFTHEDOORWAYWASREMOVEDWITHTREMBLINGHANDSIMADEATINYBREACHINTHEUPPERLEFTHANDCORNERANDTHENWIDENINGTHEHOLEALITTLEIINSERTEDTHECANDLEANDPEEREDINTHEHOTAIRESCAPINGFROMTHECHAMBERCAUSEDTHEFLAMETOFLICKERBUTPRESENTLYDETAILSOFTHEROOMWITHINEMERGEDFROMTHEMISTXCANYOUSEEANYTHINGQ'

## Etymology of missspellings
  
An anagram of the misspellings and their respective counterparts is LAQUEO.	

- **Laqueo** (Latin): a noose, snare or trap
- **Laqueo** (Spanish):
  - Lacquer. A type of varnish applied to wood
  - First person singular present indicative form of Laquear
- **Laquear** (English): (plural laquears) (architecture) A lacunar.
- **Lacunar** (medicine) Of or pertaining to a lacuna
- **Lacuna** (plural lacunae or lacunas)
  - A small opening; a small pit or depression; a small blank space; a gap or vacancy; a hiatus.
  - An absent part, especially in a book or other piece of writing, often referring to an ancient manuscript or similar such.
  - (microscopy) A space visible between cells, allowing free passage of light.
  - (linguistics) A language gap, which occurs when there is no direct translation in the target language for a lexical term found in the source language.

## Additional
The letters YAR in the ciphertext (2nd panel, first line) are raised. Whilst this may not be relevant to k4, there is an outside chance that this might be part of the solution

## K4
Workings for K4 (as yet unsolved)

One must be very careful with K4 as the whole point of cryptography is mis-direction. There have been several clues given on this cryptogram so far, including:

* The words `BERLINCLOCK` as being the plaintext for `NYPVTTMZFPK` (chars 64-75)
* That one should look in detail at that clock (widely assumed to be mengenlehruer)
* That there are many interesting clocks in Berlin
* That there is a 1-1 mapping between the letters and the plaintext (Suspect not)

My assumption has always been it's less about Mengenlehruer, and more about set theory; the basis for a number of crypto-analysis techniques

> ### Note:
> _The Vigenere grid contains 30 characters in the alphabet (`KRYPTOSABCDEFGHIJLMNQUVWXZK`).
My assumption all along is that there is only a requirement for the 26 standard characters. This, I believe is wrong but again; misdirection?_

In a speach, the sculptor also said there are several interesting clocks in berlin. Question, is this again a clue? Or more misdirection?

Of the other clocks, there are two that stand out.

* The water clock in the Europa centre
* The world clock in Alexanderplatz

Alongside this, there is another *clock* of interest.

The clock was a deciphering mechanism first developed by the Polish to crack Enigma. It would be of no suprise if Enigma was an interim cipher for K4.


## Method to add a keyword at the beginning of an alphabet

In [11]:
def apply_keyword(keyword=''):
    alphabet='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return '{}{}'.format(
        keyword,
        ''.join(
            sorted(list(set(list(alphabet)) - set(list(keyword))))
        )
    )

## Create Vigenere grid

In [12]:
def create_aphagrid(keyword='', dim=(26,26)):
    alphabet = apply_keyword(keyword)
    while len(alphabet) < dim[0]:
        alphabet += alphabet
    alphabet = alphabet[:dim[0]]
    grid     = [list(alphabet)]
    for i in range(len(grid[0])-1):
        grid += [grid[i][1:] + [grid[i][0]]]
    return grid

## Draw up viginere grid using Pandas

In [13]:
def grid(grid):
    df = pd.DataFrame()
    df = df.append(grid)
    alphabet=apply_keyword()
    while len(alphabet) < len(grid[0]):
        alphabet += alphabet
    alphabet = alphabet[:len(grid[0])]
    df.index = df.columns = pd.MultiIndex.from_tuples([
        t for t in zip(
            [x for x in range(1,len(grid[0])+1)],
            [x for x in range(len(grid[0]), -1, -1)],
            list(alphabet)
        )
    ])
    return df

In [14]:
import operator
def opsfunc(op):
    """
    The operator function allows for a string to be used as an operator
    This is then mapped to the actual method allowing it to be used as a function
    
    Only provides +,-,* as division is invalid for a ciphertext
    """
    ops = {
        "+": operator.add,
        "-": operator.sub,
        "*": operator.mul
    }
    return ops[op]

def distances(cipher, plain, alphabet='', op='+'):
    """
    Calculates the character used to get from a ciphertext character to a plaintext character
    """
    return_val = []
    for x in range(len(plain)):
        y = alphabet.index(plain[x].upper())
        z = alphabet.index(cipher[x].upper())
        
        a = (opsfunc(op)(y, z) % len(alphabet))
        return_val.append(alphabet[(opsfunc(op)(z, a) % len(alphabet))])
    return return_val

def calc(c1, c2, alphabet='', op='+'):
    """
    Calculates the distance between two characters
    """
    a1 = alphabet.index(c1)
    a2 = alphabetindex(c2)
    return opsfunc(op)(a1, a2)

In [15]:
def add(a, b):
    global decipher
    x = decipher.get_index(a)
    y = decipher.get_index(b)
    return (x + y) % 26

def subtract(a, b):
    global decipher
    x = decipher.get_index(a)
    y = decipher.get_index(b)
    return (x - y) % 26

def rotate_char(character, shift):
    character = character.upper()
    index = 0
    i = ord(character)
    while index < shift:
        i += 1
        if i > ord('Z'):
            i -= 26
        elif i < ord('A'):
            i = ord('Z') - i
        index += 1
    
    return chr(i)

In [16]:
rotate_char('Q', -1)

'Q'

## Class to determine a square on the grid

In [17]:
class SquareX(object):
    forward_x   = 0
    forward_y   = 0
    backwards_x = 0
    backwards_y = 0
    character   = ''
    index       = 0
    position    = 0
    
    property
    def top_right(self):
        return (),(forward_y, (len(decipher.actual_alphabet) - self.forward_y)-1)
    
    def __init__(self, position=1, ciphertext=None, alphabet=None):
        if ciphertext is None:
            ciphertext = decipher.ciphertext
        if alphabet is None:
            alphabet=decipher.actual_alphabet

        self.position = position
        self.character = ciphertext[self.position]
        self.index = decipher.a2i(self.character)
        
        self.forward_x = (
            self.position - self.index
        ) % len(alphabet)
        
        self.forward_x = self.forward_x if self.forward_x >= 1 else decipher.a2i('Z')
        
        self.forward_y = (
            (
                (decipher.a2i('Z') - self.forward_x) + 1 # add 1 to account for 0 index
            ) + self.index) % decipher.a2i('Z')
        
        self.backwards_x = self.forward_y
        self.backwards_y = self.forward_x

# K4 Test area

It is my belief that the k4 cipher is formed of 97 wheels, one for each character. Some of the wheels may repeat but there is no guarantee of this.

The work below shows much of this may be true, and how these wheels are formed. This moves the issue from one of deciphering to one of selecting. How does one select the wheels?

## ciphertext

The ciphertext for k4 is as follows:

In [18]:
ciphertext = 'OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR'

## clues given and their match to the letters

Over the years, the following plaintext has been provided.

- November 2010 Characters 64 - 69 (`NYPVTT`) deciphers to the word `BERLIN`
- November 2014 Characters 70 - 74 (`MZFPK`) deciphers to the word `CLOCK`
- January 2020 Characters 26 - 34 (`QQPRNGKSS`) deciphers to the word `NORTHEAST`

These go into a list with their respective ciphertext and position numbers. This is our attack vector on the ciphertext.

In [19]:
cipher_plain_pairs = [
    ('Q', 'N', 26),
    ('Q', 'O', 27),
    ('P', 'R', 28),
    ('R', 'T', 29),
    ('N', 'H', 30),
    ('G', 'E', 31),
    ('K', 'A', 32),
    ('S', 'S', 33),
    ('S', 'T', 34),
    ('N', 'B', 64),
    ('Y', 'E', 65),
    ('P', 'R', 66),
    ('V', 'L', 67),
    ('T', 'I', 68),
    ('T', 'N', 69),
    ('M', 'C', 70),
    ('Z', 'L', 71),
    ('F', 'O', 72),
    ('P', 'C', 73),
    ('K', 'K', 74)
]

The way to attack this cipher is to look at what is required to get from the ciphertext to the plaintext given above.

First, we need to define two functions. One calculates the addition vector to move forward from the ciphertext to the plaintext, the second defines a subtraction vector for the same.

For example, to move forward through the alphabet from character N to character D, we pass through Z and wrap round giving us 

`(Z - N) + D = (26 - 14) + 4 = 13 + 4 = 17 = Q`

The subtraction vector is simply the inverse of this.

In [20]:
def distanceto(x, y):
    """
    Addition vector moving through Z
    
    :param x The starting character
    :param y The character to calculate the distance to
    
    :return char
    """
    a = decipher.a2i(x)
    b = decipher.a2i(y)
    c = (26 - a) + b
    return decipher.i2a(c % 26)

def distancefrom(x, y):
    """
    Subtraction vector moving through Z
    
    :param x The starting character
    :param y The character to calculate the distance to
    
    :return char
    """
    a = decipher.a2i(x)
    b = decipher.a2i(y)
    
    c = ((26 - a) - b)
    return decipher.i2a(c % 26)

Using these functions we can then create a new table to work the ciphertext.

This table contains the following columns:

1. Ciphertext character
2. Position in the overall ciphertext
3. Respective index of a character in an alphabet repeating the length of the ciphertext
4. Inverse alphabetical character index (Alphabet starts at end of ciphertext and runs backwards)
5. Character showing `distancefrom` ciphertext to plaintext
6. Plaintext character calculated using `distancefrom` the value above
7. Character showing `distanceto` ciphertext to plaintext
8. Inverse of plaintext version above

In [21]:
distance_table = [
    (
        x[0], # 1.
        x[2], # 2.
        str(  # 3.
            (x[2] % 26) if x[2] % 26 != 0 else 26
        ).zfill(2),
        
        str( # 4.
            ((97 - x[2]) % 26) if (97 - x[2]) % 26 != 0 else 26
        ).zfill(2),

        # negative from cipher to plain
        distancefrom(x[0], x[1]),                      # 5.
        distancefrom(x[0], distancefrom(x[0], x[1]),), # 6.
        distanceto(x[0], distancefrom(x[0], x[1])),    # 7.
        distanceto(x[0], distanceto(x[0], x[1]),),     # 8.
    )
    for x in cipher_plain_pairs
]
distance_table

[('Q', 26, '26', '19', 'U', 'N', 'D', 'F'),
 ('Q', 27, '01', '18', 'T', 'O', 'C', 'G'),
 ('P', 28, '02', '17', 'R', 'R', 'B', 'L'),
 ('R', 29, '03', '16', 'N', 'T', 'V', 'J'),
 ('N', 30, '04', '15', 'D', 'H', 'P', 'F'),
 ('G', 31, '05', '14', 'N', 'E', 'G', 'Q'),
 ('K', 32, '06', '13', 'N', 'A', 'C', 'E'),
 ('S', 33, '07', '12', 'N', 'S', 'U', 'G'),
 ('S', 34, '08', '11', 'M', 'T', 'T', 'H'),
 ('N', 64, '12', '07', 'J', 'B', 'V', 'Z'),
 ('Y', 65, '13', '06', 'V', 'E', 'W', 'G'),
 ('P', 66, '14', '05', 'R', 'R', 'B', 'L'),
 ('V', 67, '15', '04', 'R', 'L', 'V', 'T'),
 ('T', 68, '16', '03', 'W', 'I', 'C', 'U'),
 ('T', 69, '17', '02', 'R', 'N', 'X', 'Z'),
 ('M', 70, '18', '01', 'J', 'C', 'W', 'C'),
 ('Z', 71, '19', '26', 'N', 'L', 'N', 'L'),
 ('F', 72, '20', '25', 'E', 'O', 'Y', 'C'),
 ('P', 73, '21', '24', 'G', 'C', 'Q', 'W'),
 ('K', 74, '22', '23', 'D', 'K', 'S', 'O')]

In the table above, we can see that in order to get from QQPRNGKSS to the word NORTHEAST, we have to either pass through UTRNDNNNM or DCBVPGUTV

This gives a potential attack vector in that we can look at the distances from QQPRNGKSS to UTRNDNNNM

## Wheel table

In [22]:
#c = ''.join([c[0] for c in distance_table])
#s  = ''.join([c[4] for c in distance_table])#

c = 'QQPRNGKSSNYPVTTMZFPK'
s = 'RRTPXLDNNXBTHLLZZNTD'
# this is backwards but easier to set up.
# each entry in spokes is a single spoke set from
# multiple wheels, with the entire spoke set forming 
# the whole wheel. 
spokes = []

for i in range(6):
    z = ''
    y = []
    for j in range(len(c)):
        z += distancefrom(c[j], s[j])
    print('{} {} {} {}'.format(str(i).zfill(2), c, s, z))
    spokes.append(c)

    c = s
    s = z

00 QQPRNGKSSNYPVTTMZFPK RRTPXLDNNXBTHLLZZNTD QQPRNGKSSNYPVTTMZFPK
01 RRTPXLDNNXBTHLLZZNTD QQPRNGKSSNYPVTTMZFPK QQPRNGKSSNYPVTTMZFPK
02 QQPRNGKSSNYPVTTMZFPK QQPRNGKSSNYPVTTMZFPK RRTPXLDNNXBTHLLZZNTD
03 QQPRNGKSSNYPVTTMZFPK RRTPXLDNNXBTHLLZZNTD QQPRNGKSSNYPVTTMZFPK
04 RRTPXLDNNXBTHLLZZNTD QQPRNGKSSNYPVTTMZFPK QQPRNGKSSNYPVTTMZFPK
05 QQPRNGKSSNYPVTTMZFPK QQPRNGKSSNYPVTTMZFPK RRTPXLDNNXBTHLLZZNTD


What we see here is a wheel moving in 6 phases from QQPRNG... to VWXDJSW with duplication of the value of `distanceto` calculated between the ciphertext and the plaintext above.

Curiously, we also see, in the second position (horizontally or vertically) a number of repeated characters.

Using this as a basis to start, we calculate an alphabet table merging each character of the ciphertext with each letter of the alphabet, such that `OBKR` is masked with `AAAA`, `BBBB`, `CCCC`, etc.

## Z-mappings table

In [23]:
alphabet = apply_keyword('')
translate_table_from = []

translate_df = []
for c in alphabet:
    translate_df.append(
        [ 
            distancefrom(ciphertext[i], c) for i in range(len(ciphertext))
        ]
    )
    translate_table_from.append(
        ''.join([ 
            distancefrom(ciphertext[i], c) for i in range(len(ciphertext))
        ])
    )
translate_df = pd.DataFrame(translate_df)
translate_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
0,J,W,N,G,D,J,A,J,R,Q,D,M,W,F,J,M,P,S,W,W,B,S,M,G,C,H,H,I,G,K,R,N,F,F,J,E,B,E,H,F,O,H,F,F,T,N,Y,Y,B,X,E,O,N,M,D,U,P,X,B,P,K,S,W,K,Z,I,C,E,E,L,Y,S,I,N,B,R,U,N,Y,A,E,O,V,U,P,R,N,D,Q,D,X,D,T,N,V,X,G
1,I,V,M,F,C,I,Z,I,Q,P,C,L,V,E,I,L,O,R,V,V,A,R,L,F,B,G,G,H,F,J,Q,M,E,E,I,D,A,D,G,E,N,G,E,E,S,M,X,X,A,W,D,N,M,L,C,T,O,W,A,O,J,R,V,J,Y,H,B,D,D,K,X,R,H,M,A,Q,T,M,X,Z,D,N,U,T,O,Q,M,C,P,C,W,C,S,M,U,W,F
2,H,U,L,E,B,H,Y,H,P,O,B,K,U,D,H,K,N,Q,U,U,Z,Q,K,E,A,F,F,G,E,I,P,L,D,D,H,C,Z,C,F,D,M,F,D,D,R,L,W,W,Z,V,C,M,L,K,B,S,N,V,Z,N,I,Q,U,I,X,G,A,C,C,J,W,Q,G,L,Z,P,S,L,W,Y,C,M,T,S,N,P,L,B,O,B,V,B,R,L,T,V,E
3,G,T,K,D,A,G,X,G,O,N,A,J,T,C,G,J,M,P,T,T,Y,P,J,D,Z,E,E,F,D,H,O,K,C,C,G,B,Y,B,E,C,L,E,C,C,Q,K,V,V,Y,U,B,L,K,J,A,R,M,U,Y,M,H,P,T,H,W,F,Z,B,B,I,V,P,F,K,Y,O,R,K,V,X,B,L,S,R,M,O,K,A,N,A,U,A,Q,K,S,U,D
4,F,S,J,C,Z,F,W,F,N,M,Z,I,S,B,F,I,L,O,S,S,X,O,I,C,Y,D,D,E,C,G,N,J,B,B,F,A,X,A,D,B,K,D,B,B,P,J,U,U,X,T,A,K,J,I,Z,Q,L,T,X,L,G,O,S,G,V,E,Y,A,A,H,U,O,E,J,X,N,Q,J,U,W,A,K,R,Q,L,N,J,Z,M,Z,T,Z,P,J,R,T,C
5,E,R,I,B,Y,E,V,E,M,L,Y,H,R,A,E,H,K,N,R,R,W,N,H,B,X,C,C,D,B,F,M,I,A,A,E,Z,W,Z,C,A,J,C,A,A,O,I,T,T,W,S,Z,J,I,H,Y,P,K,S,W,K,F,N,R,F,U,D,X,Z,Z,G,T,N,D,I,W,M,P,I,T,V,Z,J,Q,P,K,M,I,Y,L,Y,S,Y,O,I,Q,S,B
6,D,Q,H,A,X,D,U,D,L,K,X,G,Q,Z,D,G,J,M,Q,Q,V,M,G,A,W,B,B,C,A,E,L,H,Z,Z,D,Y,V,Y,B,Z,I,B,Z,Z,N,H,S,S,V,R,Y,I,H,G,X,O,J,R,V,J,E,M,Q,E,T,C,W,Y,Y,F,S,M,C,H,V,L,O,H,S,U,Y,I,P,O,J,L,H,X,K,X,R,X,N,H,P,R,A
7,C,P,G,Z,W,C,T,C,K,J,W,F,P,Y,C,F,I,L,P,P,U,L,F,Z,V,A,A,B,Z,D,K,G,Y,Y,C,X,U,X,A,Y,H,A,Y,Y,M,G,R,R,U,Q,X,H,G,F,W,N,I,Q,U,I,D,L,P,D,S,B,V,X,X,E,R,L,B,G,U,K,N,G,R,T,X,H,O,N,I,K,G,W,J,W,Q,W,M,G,O,Q,Z
8,B,O,F,Y,V,B,S,B,J,I,V,E,O,X,B,E,H,K,O,O,T,K,E,Y,U,Z,Z,A,Y,C,J,F,X,X,B,W,T,W,Z,X,G,Z,X,X,L,F,Q,Q,T,P,W,G,F,E,V,M,H,P,T,H,C,K,O,C,R,A,U,W,W,D,Q,K,A,F,T,J,M,F,Q,S,W,G,N,M,H,J,F,V,I,V,P,V,L,F,N,P,Y
9,A,N,E,X,U,A,R,A,I,H,U,D,N,W,A,D,G,J,N,N,S,J,D,X,T,Y,Y,Z,X,B,I,E,W,W,A,V,S,V,Y,W,F,Y,W,W,K,E,P,P,S,O,V,F,E,D,U,L,G,O,S,G,B,J,N,B,Q,Z,T,V,V,C,P,J,Z,E,S,I,L,E,P,R,V,F,M,L,G,I,E,U,H,U,O,U,K,E,M,O,X


As we know there are repetitions of the letter `N` which map to positions 31 to 33, we can confirm at this position we get the `EAS` of the word `NORTHEAST`. However this is not the only interesting thing about this table.

Looking in the wheel map above, we see the value `IIJHLSOGGLAJDFFMZTJO`

Splitting this into the respective words NORTHEAST BERLIN CLOCK we get `IIJHLSOGG LAJDFF MZTJO`

This is directly mapped against `Z` in the table above in the same positions. This tells us that there is `Z` mapping on every value in the wheel, creating pairs. We can prove this by taking the individual elements of the wheel table and applying `Z` to it.

In [24]:
for spoke in spokes:
    print('{} = {}'.format(
        spoke,
        ''.join([distancefrom(c, 'Z') for c in spoke])
    ))

QQPRNGKSSNYPVTTMZFPK = IIJHLSOGGLAJDFFMZTJO
RRTPXLDNNXBTHLLZZNTD = HHFJBNVLLBXFRNNZZLFV
QQPRNGKSSNYPVTTMZFPK = IIJHLSOGGLAJDFFMZTJO
QQPRNGKSSNYPVTTMZFPK = IIJHLSOGGLAJDFFMZTJO
RRTPXLDNNXBTHLLZZNTD = HHFJBNVLLBXFRNNZZLFV
QQPRNGKSSNYPVTTMZFPK = IIJHLSOGGLAJDFFMZTJO


This shows that each wheel is formed from 3 pairs with each pair being formed from a character, and that characters distance from Z.

If we look deeper into this, we can also determine that there is a secong position on the wheel which decipher to the same plaintext, this being the difference between `IIJHLSOGGLAJDFFMZTJO` and `EFHLVLLLMPDHHCHPLUSV` which forms a new character set `LKHFRUYGFXUHNQLWNKWO`, the reflection of which is also `NORTHEASTBERLINCLOCK`.

In [25]:
c = 'IIJHLSOGGLAJDFFMZTJO'
s = 'EFHLVLLLMPDHHCHPLUSV'

# First, calculate the inflection point
reflector = ''
for j in range(len(c)):
    reflector += distancefrom(c[j], s[j])

print('Reflection point: {}\n'.format(reflector))

# Now set c as being the inflection and reset the inflector
c = reflector
for i in range(12):
    z = ''
    for j in range(len(c)):
        z += distanceto(c[j], s[j])
    print('{}'.format(z))
    c = s
    s = z

Reflection point: LKHFRUYGFXUHNQLWNKWO

SUZFDQMEGRIZTLVSXJVG
NORTHEASTBERLINCLOCK
UTRNDNNNMJVRRWRJNEGD
GEZTVIMUSHQZFNDGBPDS
LKHFRUYGFXUHNQLWNKWO
EFHLVLLLMPDHHCHPLUSV
SUZFDQMEGRIZTLVSXJVG
NORTHEASTBERLINCLOCK
UTRNDNNNMJVRRWRJNEGD
GEZTVIMUSHQZFNDGBPDS
LKHFRUYGFXUHNQLWNKWO
EFHLVLLLMPDHHCHPLUSV


This covers the differences between the wheel pairs `QQPRNGKSSNYPVTTMZFPK` and `UTRNDNNNMJVRRWRJNEGD` along with their associated reflections. The remaining wheel and it's associated reflection can be used to calculate the ciphertext.

## Creating the wheels

From this, we can now start to construct all the available wheels for the ciphertext.

This is achieved by first creating an entire alphabet for each character of the ciphertext, then determining the inflection points.

For example, taking the letter Q, we write this out as pairs as follows:

```
QA
QB
QC
QD
...
```

Then we calculate from this, the remaining spokes on each wheel.

In [26]:
from itertools import product
from copy import deepcopy

class Wheel(object):
    _character = ''
    _spokes    = None
    _shift     = 1
    
    def __init__(self, character, spoke):
        self._spokes = []
        self._character = character
        self._spokes.append(character)
        self._spokes.append(spoke)
        
        self._spokes.append(
            distanceto(character, spoke)
        )
        self._spokes.append(
            distancefrom(character, 'Z')
        )
        self._spokes.append(
            distanceto(spoke, 'Z')
        )
        self._spokes.append(
            distancefrom(self.spokes[2], 'Z')
        )
    
    @property
    def character(self):
        return self._character
    
    @property
    def spokes(self):
        return self._spokes
    
    @property
    def step(self):
        return self._shift
    
    @step.setter
    def step(self, value):
        self._shift = value
    
    @property
    def cipher(self):
        return self.spokes[0]
    
    @property
    def translate(self):
        return self.spokes[1]
        
def create_wheels(char, index):
    """
    Create a character wheel
    
    :param: char  - The character to create the wheels for
    :param: index - The characters position in the ciphertext
    
    :return: List of tuples
    """
    r = distancefrom(char, 'Z')

    pairs = []
    i = 0
    for element in list(product(char, ''.join(decipher.actual_alphabet), repeat=1)):
        # ignore any that don't start with the current letter
        if (element[0] != char):
            continue
        
        pairs.append(Wheel(element[0], element[1]))
        i+=1
    return pairs

f = ''
i = 1
for q in ciphertext:
    pairs = create_wheels(q, i)
    # For now I'm just going to target the ciphertext for which we have the
    # associated plaintext
    if i in range(26,34) or i in range(64,75):
        _ = [print(p.spokes) for p in pairs]
        print()
    i += 1


['Q', 'A', 'J', 'I', 'Y', 'P']
['Q', 'B', 'K', 'I', 'X', 'O']
['Q', 'C', 'L', 'I', 'W', 'N']
['Q', 'D', 'M', 'I', 'V', 'M']
['Q', 'E', 'N', 'I', 'U', 'L']
['Q', 'F', 'O', 'I', 'T', 'K']
['Q', 'G', 'P', 'I', 'S', 'J']
['Q', 'H', 'Q', 'I', 'R', 'I']
['Q', 'I', 'R', 'I', 'Q', 'H']
['Q', 'J', 'S', 'I', 'P', 'G']
['Q', 'K', 'T', 'I', 'O', 'F']
['Q', 'L', 'U', 'I', 'N', 'E']
['Q', 'M', 'V', 'I', 'M', 'D']
['Q', 'N', 'W', 'I', 'L', 'C']
['Q', 'O', 'X', 'I', 'K', 'B']
['Q', 'P', 'Y', 'I', 'J', 'A']
['Q', 'Q', 'Z', 'I', 'I', 'Z']
['Q', 'R', 'A', 'I', 'H', 'Y']
['Q', 'S', 'B', 'I', 'G', 'X']
['Q', 'T', 'C', 'I', 'F', 'W']
['Q', 'U', 'D', 'I', 'E', 'V']
['Q', 'V', 'E', 'I', 'D', 'U']
['Q', 'W', 'F', 'I', 'C', 'T']
['Q', 'X', 'G', 'I', 'B', 'S']
['Q', 'Y', 'H', 'I', 'A', 'R']
['Q', 'Z', 'I', 'I', 'Z', 'Q']

['Q', 'A', 'J', 'I', 'Y', 'P']
['Q', 'B', 'K', 'I', 'X', 'O']
['Q', 'C', 'L', 'I', 'W', 'N']
['Q', 'D', 'M', 'I', 'V', 'M']
['Q', 'E', 'N', 'I', 'U', 'L']
['Q', 'F', 'O', 'I', 'T', 'K']
['Q', '

['F', 'D', 'X', 'T', 'V', 'B']
['F', 'E', 'Y', 'T', 'U', 'A']
['F', 'F', 'Z', 'T', 'T', 'Z']
['F', 'G', 'A', 'T', 'S', 'Y']
['F', 'H', 'B', 'T', 'R', 'X']
['F', 'I', 'C', 'T', 'Q', 'W']
['F', 'J', 'D', 'T', 'P', 'V']
['F', 'K', 'E', 'T', 'O', 'U']
['F', 'L', 'F', 'T', 'N', 'T']
['F', 'M', 'G', 'T', 'M', 'S']
['F', 'N', 'H', 'T', 'L', 'R']
['F', 'O', 'I', 'T', 'K', 'Q']
['F', 'P', 'J', 'T', 'J', 'P']
['F', 'Q', 'K', 'T', 'I', 'O']
['F', 'R', 'L', 'T', 'H', 'N']
['F', 'S', 'M', 'T', 'G', 'M']
['F', 'T', 'N', 'T', 'F', 'L']
['F', 'U', 'O', 'T', 'E', 'K']
['F', 'V', 'P', 'T', 'D', 'J']
['F', 'W', 'Q', 'T', 'C', 'I']
['F', 'X', 'R', 'T', 'B', 'H']
['F', 'Y', 'S', 'T', 'A', 'G']
['F', 'Z', 'T', 'T', 'Z', 'F']

['P', 'A', 'K', 'J', 'Y', 'O']
['P', 'B', 'L', 'J', 'X', 'N']
['P', 'C', 'M', 'J', 'W', 'M']
['P', 'D', 'N', 'J', 'V', 'L']
['P', 'E', 'O', 'J', 'U', 'K']
['P', 'F', 'P', 'J', 'T', 'J']
['P', 'G', 'Q', 'J', 'S', 'I']
['P', 'H', 'R', 'J', 'R', 'H']
['P', 'I', 'S', 'J', 'Q', 'G']
['P', '

At this point the problem of deciphering K4 moves from deciphering to selecting. We know *how* to decipher the message but we don't know how to select the wheels.

What is interesting about these wheels is each character set appears to be presented in 2 sets which we could label A and B.

In the code block above, we create variable `d` as 
```
d = (
    (
        decipher.a2i(element[1]) -
        decipher.a2i(x) -
        decipher.a2i(b) -
        decipher.a2i(c)
    ) - (97-index)
) % 26
```

This creates a revolving index, pivoting on 26 (Z), defining the two sets. This can be equated to the rotar set in enigma with an additional complexity of for every character in the alphabet there are 26 wheel sets split into two groups pivoting on M in the second column.

## Observations towards deciphering the key

Given the plaintext that has been provided, there are indications that a sequential pattern is moving down through the ciphertext message which forms the basis of the key.

This pattern is visible in a number of places, particularly where there are repetitions of characters (ciphertext) or plaintext repetitions in close proximity.

We see this in the letters `QQ`, `SS` and the C's in `CLOCK`.

Taking the `Q`s first, we note that these decipher to `NO` from `NORTHEAST`. `Q` is the 17th character of the standard alphabet whilst `N` and `O` are the 14th and 15th characters respectively.

Similarly, the `SS`s play a similar role deciphering to the 19th and 20th characters of the alphabet. Co-incidentally, these also map to sequential rows when translating through `Z` as we see in the [Z-mappings table](#Z-mappings-table) above.

With the `C`s from `CLOCK`, we note that the letters M and P are both spaced by 2 characters with M being the 13th character and P being the 16th.

We can deduce from this that as the ciphertext moves forward by 1 character, associated wheels turn by 1 character.

This could feed into the wheel selection process, for example, for any given starting offset, we move the selection position for all wheels by 1 character forwards or backwards

**Example:**

Given an offset of 4 and a ciphertext of HELLOWORLD, we would select wheel 4 from the H position. Moving to character E, we would select the 5th wheel, L the 6th and so on and so forth.

This does not hold entirely true for deciphering. We know from the intermediate characters used to decipher the plaintext (`UTRNDNNNMJVRRWRJNEGD`), where we have repeated characters, the associated wheels are all wheel 14 (`N`) from the character wheels for `R, G, K S and Z`, wheel 18 (`R`) or wheel 4 (`D`)

I say *entirely true*... The wheel selection doesn't seem to hold for repeated letters in the intermediate ciphertext, but what about where there are repeated letters in the ciphertext or plaintext?

- `QQ` deciphers to the intermediate step `UT` with T being 1 character prior to U.
- `SS` deciphers to the intermediate step `NM` with M being 1 character prior to M
- `CloCk` has the characters `CC`, associated ciphertext `MP` and the intermediate equivelants of `JG`. G is separated from J by 2 characters.

The difference here is that the alphabets on the intermediate step are running backwards, this masks the sequential nature of the cipher a litte.

> **Note:**
>
> This is clearer if you use the reflection of `UTRNDNNNMJVRRWRJNEGD` as this moves forward through the alphabet, not backwards.

There are a number of questions here which need answering such as:

- Do all wheels start with the same offset?
- Is there even an offset or am I being presumtious?
- Does the offset on all wheels move with each character in the ciphertext or do they only change when that character has been hit?

The indication is that there is an offset but that it is perhaps set differently at the start of the cipher for each character of the alphabet. Given the changes between how characters encipher as shown above, the suggestion is furthered that all characters change wheels with each character of the ciphertext. This is backed up by the difference in the C's which are, always 4 characters apart forwards or backwards.

If the wheels all start at different offsets, there are 26 wheels to each letter of the alphabet. As we know the order in which the characters of the alphabet will be entered, that still leaves 26\*26 different starting positions to test, however this is reduced by half once you take the known characters into consideration.

For the known characters, we can determine their offset by working backwards from their first known position to the start of the string. We ignore duplicated characters as these are not relevant whilst missing characters are filled with the character `A`

In [27]:
def create_known_positions(
    ciphertext,
    substitution,
    start_index=25,
    secondary_index=63,
    jump_at=33,
    fill=True
):
    positions = {}

    ps = ''
    for i in range(len(ciphertext)):
        char = ciphertext[i]
        spoke = substitution[i]
        
        # check if we have created this position already
        if char not in positions.keys():
            pos = decipher.a2i(spoke)
            
            # rewind from the position back to 0
            # loop the position on 0 back to 26 ('Z')
            for _ in range(start_index, 0, -1):
                pos = (pos - 1) if (pos - 1) > 0 else 26
            
            # store this position
            positions[char] = pos

        if jump_at and start_index == jump_at:
            start_index = secondary_index
        else:
            start_index += 1

    positions = {k: decipher.i2a(v) for k, v in positions.items()}

    if fill:
        # Now fill the missing characters with 'A'
        for c in decipher.actual_alphabet:
            if c not in positions.keys():
                positions[c] = 'A'
    return positions #dict(sorted(positions.items()))

create_known_positions('QQPRNGKSSNYPVTTMZFPK', 'EFHLVLLLMPDHHCHPLUSV')

{'Q': 'F',
 'P': 'G',
 'R': 'J',
 'N': 'S',
 'G': 'H',
 'K': 'G',
 'S': 'F',
 'Y': 'R',
 'V': 'T',
 'T': 'N',
 'M': 'Y',
 'Z': 'T',
 'F': 'B',
 'A': 'A',
 'B': 'A',
 'C': 'A',
 'D': 'A',
 'E': 'A',
 'H': 'A',
 'I': 'A',
 'J': 'A',
 'L': 'A',
 'O': 'A',
 'U': 'A',
 'W': 'A',
 'X': 'A'}

In [28]:
positions = create_known_positions(
    'QQPRNGKSSNYPVTTMZFPK',
    'EFHLVLLLMPDHHCHPLUSV',
    jump_at=33
)

step_on = ['P', 'N', 'K', ]
def step(ch, index, wheel):
    """
    Determine step factor
    
    Depending on the "step" value of the wheel selection, we get different
    words appearing.
    
    This suggests to me that there is a method to pre-determine which wheel
    should be used that is placed in conjunction witht the position set.
    
    Without this in place, most of the message is garbage but there is confirmation
    the stepping works in the following:
    
    OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
    LZXUJQISIBPZKZZDHMQRXQKOVNORTHEASTTCNEAZEDCDNONOZWRPVPHZVEJYMEIPEDLIJCLOKQZXUUTDVTBBXHDOCQLSJKNQJ
                             ^-------^                              ^ ^^ ^^^
    The section immediately following the word NORTHEAST is most likely either "I ENGAGED" or "HE GAZED".
    
    By changing the step value to ?? we can reveal the whole of the word CLOCK
    """
    s = wheel[8]
    c = decipher.a2i(ch)
    stepvalue = (c + s) % 26 if (c + s) % 26 != 0 else 1
    return stepvalue

print(ciphertext)
plain = ''
for i in range(len(ciphertext)):
    # This area is problematic. See step function above
    pos = positions[ciphertext[i]]

    wheel = [
        w for w in create_wheels(
            ciphertext[i], (i + 1)
        ) if w.spokes[4] == pos
    ][0]

    step = 1
    z_at = 26
    positions = {
        k: decipher.i2a(
            (decipher.a2i(v) + step)
            if (decipher.a2i(v) + step) <= z_at else 1# loop back round
        )
        for k, v in positions.items()
    }
    plain += distancefrom(wheel.cipher, wheel.translate)
print(plain)

OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
LZXUJQISIBPZKZZDHMQRXQKOVNORTHEASTTCNEAZEDCDNONOZWRPVPHZVEJYMEIPEDLIJCLOKQZXUUTDVTBBXHDOCQLSJKNQJ


We see something of a translation in the method above.

There is certainly a pattern beginning to form whereby the correct plaintext is being revealed at the correct positions.

Of course some of this could be circumstancial but my feeling is not.

The determination of initial positions, followed by stepping them up as the ciphertext moves, character by character definitely holds merit but doesn't come cleanly.

It seems to be that the underlying principle of executing the machine in this manner is correct and valid but there is clearly a disjoint between the alphabet created and how the machine steps.

It could be that we require a full 97 character alphabet, in which case there is reason to discover how the alphabet is constructed. If not, then there must be a factor which determines how each wheel steps.

To look at the two alphabets for the known positions:

In [29]:
''.join(create_known_positions(
    'QQPRNGKSS',
    'EFHLVLLLM',
    start_index=25,
    fill=False
).values())

'FGJSHGF'

In [30]:
''.join(create_known_positions(
    'NYPVTTMZFPK',
    'PDHHCHPLUSV',
    start_index=63,
    fill=False
).values())

'ERUTNYTBA'

Now, in the two outputs above, we have an alphabet that is slightly shorter than the original ciphertext in the same position.

We can fill in the missing characters according to the stepping facility we designed above, which gives 

`FGGJSHGFG ERUTNOYTBVA`

If we place that inline with the message, filling in the missing characters with the letter `A`, then run the machine, we do not quite get what we expect...

In [31]:
plain = ''
positions = 'AAAAAAAAAAAAAAAAAAAAAAAAAFGGJSHGFGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAERUTNOYTBVAAAAAAAAAAAAAAAAAAAAAAAA'
for i in range(len(ciphertext)):
    pos = positions[i]

    wheel = [
        w for w in create_wheels(
            ciphertext[i], (i + 1)
        ) if w.spokes[4] == pos
    ][0]

    step = 1
    z_at = 26
    positions = ''.join(
        decipher.i2a(
            (decipher.a2i(v) + step)
            if (decipher.a2i(v) + step) <= z_at else 1# loop back round
        )
        for v in positions
    )
    plain += distancefrom(wheel.cipher, wheel.translate)
print(plain)

LZRLJQISBBPZKUZDHLQRXPKFCNPRTHEASUTPNRVUEYXYNIUVZWEPPPHZVEJYUDIBERLIKCLOZKZQUOADITBBXAXOCQLSJENQA


Note the difference?

Instead of `NORTHEAST` we get `NPRTHEASU` and instead of `BERLINCLOCK` we get `BERLIKCLOZK`. Small differences, easily corrected.

What we end up with instead is:

```
QQPRNGKSS NYPVTT MZFPK
FFGJSHGFF ERUTNR YTBYA
NORTHEAST BERLIN CLOCK
```

This discrepancy was due to the fact the machine steps the alphabet so stepping it by hand in creating the initial positions almost certainly let us down.

In [32]:
plain = ''
positions = 'AAAAAAAAAAAAAAAAAAAAAAAAAFFGJSHGFFAAAAAAAAAAAAAAAAAAAAAAAAAAAAAERUTNRYTBYAAAAAAAAAAAAAAAAAAAAAAAA'
for i in range(len(ciphertext)):
    pos = positions[i]

    wheel = [
        w for w in create_wheels(
            ciphertext[i], (i + 1)
        ) if w.spokes[4] == pos
    ][0]

    step = 1
    z_at = 26
    positions = ''.join(
        decipher.i2a(
            (decipher.a2i(v) + step)
            if (decipher.a2i(v) + step) <= z_at else 1# loop back round
        )
        for v in positions
    )
    plain += distancefrom(wheel.cipher, wheel.translate)
print(plain)

LZRLJQISBBPZKUZDHLQRXPKFCNORTHEASTTPNRVUEYXYNIUVZWEPPPHZVEJYUDIBERLINCLOCKZQUOADITBBXAXOCQLSJENQA


So now we have confirmation that the cipher works by advancing another cipher along a path then masking it. But what is this other cipher?

Question:

What cipher are we given if we simply rewind the original ciphertext by its index position?

In [33]:
sa = ''
for i in range(len(ciphertext)):
    sa += decipher.i2a(
        (decipher.a2i(ciphertext[i]) - i) % 26
    )
sa

'OAIOQJRHYYKAPFAWSOJICKPUXRQOPKCFMLGKMIEFVBCBMRFEADVKKKSAEVQBFWRCMCHEDVHMVPAJFLZWRGYYCZCLXJOHQVMJZ'

In [34]:
sb = ''
for i in range(0, len(ciphertext)):
    sb += decipher.i2a(
        (decipher.a2i(ciphertext[i]) + i) % 26
    )
sb

'OCMUYTDVOQEWNFCAYWTUQAHOTPQQTQKPYZWCGECFXFIJWDTUSXRIKMWGMFCPVOLYKCJIJDRYJFSDBJZYVMGIONSDRFMHSZSRJ'

In [35]:
print(ciphertext)
s = ''
for i in range(len(ciphertext)):
    s += distancefrom(sa[::-1][i], sb[i])
print(s)
s = ''
for i in range(len(ciphertext)):
    s += distanceto(sa[::-1][i], sb[i])
print(s)

OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
KMZIJXGTMWRCIUXRIFFSCOQUJWAMBDGGNWKZMSFXWSXERKJAFWBYBKZQQNRAQZGOBQMFZGQIRYQKUGPLKPRKUKVWILUHPKXGA
OSZYHLOLQEBWKGDTGZTIKQGYXCIUPLCMLWEFACLJSEPYLSXQRSLQXKTEQZXGIDEMXWGXTOAGLKCSYAPJCPFCYMHESXUXBKJQU


In [36]:
s = ''
for i in range(len(ciphertext)):
    s += decipher.i2a(
        (decipher.a2i(sb[i]) ^ decipher.a2i(sa[i])) % 26
    )
print(ciphertext)
print(s)

OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
ZBDZHDVDVHNVDZBVJXDBRJXZLBZDDZHVTVPHJLFZNDJHZVRPRBDBZFDFHPRRPXDZFZBLNRZTBVRNDFZNDJDPLTPHJLBZBLDXP


In [37]:
cipher    = ciphertext
positions = ciphertext[::-1]

reverse = False
use_distances = True
for _ in range(52):
    plain = ''
    for i in range(len(ciphertext)):
        pos = positions[i]

        wheel = [
            w for w in create_wheels(
                cipher[i], (i + 1)
            ) if w.spokes[4] == pos
        ][0]

        step = 1
        z_at = 26
        positions = ''.join(
            decipher.i2a(
                (decipher.a2i(v) + step)
                if (decipher.a2i(v) + step) <= z_at else 1# loop back round
            )
            for v in positions
        )

        if use_distances:
            if not reverse:
                plain += distancefrom(wheel.cipher, wheel.translate)
            else:
                plain += distanceto(wheel.cipher, wheel.translate)
        else:
            if not reverse:
                plain += decipher.i2a(
                    (decipher.a2i(wheel.cipher) + decipher.a2i(wheel.translate)) % 26
                )
            else:
                plain += decipher.i2a(
                    (decipher.a2i(wheel.cipher) - decipher.a2i(wheel.translate)) % 26
                )
                
        reverse = !reverse
    print(ciphertext)
    print(plain)
    positions = cipher
    cipher = plain

OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
CVJTVKUICNJVCPTOGEFTERUZPDIVLOSTBLAQELZSSPVDRLLDJBHFJTJBCAFPGQYHVLICXFQJTBUPANXUUADXIZLNAEOCLHVFA
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
HACKEUAUHUKGIDCJTXZKDGWKPFZMTTVPYNBFNIWAIDULKJTAWZZKESSQJSQTWTOSSKGNRPQQVRLFWBBFILOSBLUHGOXOURKBO
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
CVJTVKUICNJVCPTOGEFTERUZPDIVLOSTBLAQELZSSPVDRLLDJBHFJTJBCAFPGQYHVLICXFQJTBUPANXUUADXIZLNAEOCLHVFA
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
HACKEUAUHUKGIDCJTXZKDGWKPFZMTTVPYNBFNIWAIDULKJTAWZZKESSQJSQTWTOSSKGNRPQQVRLFWBBFILOSBLUHGOXOURKBO
OBKRUOXOGHULBSOLIFBB

OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
CVJTVKUICNJVCPTOGEFTERUZPDIVLOSTBLAQELZSSPVDRLLDJBHFJTJBCAFPGQYHVLICXFQJTBUPANXUUADXIZLNAEOCLHVFA
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
HACKEUAUHUKGIDCJTXZKDGWKPFZMTTVPYNBFNIWAIDULKJTAWZZKESSQJSQTWTOSSKGNRPQQVRLFWBBFILOSBLUHGOXOURKBO
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
CVJTVKUICNJVCPTOGEFTERUZPDIVLOSTBLAQELZSSPVDRLLDJBHFJTJBCAFPGQYHVLICXFQJTBUPANXUUADXIZLNAEOCLHVFA
OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR
HACKEUAUHUKGIDCJTXZKDGWKPFZMTTVPYNBFNIWAIDULKJTAWZZKESSQJSQTWTOSSKGNRPQQVRLFWBBFILOSBLUHGOXOURKBO
OBKRUOXOGHULBSOLIFBB

In [38]:
a = 'EFHLVLLLM'
b = 'FFGJSHGFF'

s = ''
for i in range(len(a)):
    s += distanceto(a[i], b[i])
print(s)

AZYXWVUTS


## Distance calculator method

In order to examine deeper how the wheels move, we need to define a method for generating all possibilities between two known positions.

This is effectively an extension of the machine defined above

> ### Note
>
> This method may take a couple of minutes to run. It will complete
> but needs to finish iterations of the search. Be patient.

This method has been amended on 1/06/2020 for the inclusion of two additional pieces of functionality.

1. include a flag which allows the distances to be printed as numerical values
2. An additional flag is printed at the end of each line:
  - E if all characters in the line are even characters
  - M if there is a mix of odd and even
  - O if all characters are odd
 
Testing with the functionality defined under 1 revealed a pattern which lead to the creation of point 2. Whilst I am currently unaware of the significance of this, it is intriguing as the pattern holds in most instances.

In [39]:
def is_odd_or_even(string):
    """
    Returns O if all characters in string are odd, E if all are even or M if there is a mix
    """
    return (
        'E' if all(j % 2 == 0 for j in [decipher.a2i(t) for t in string])
        else 'O' if all(not (j % 2 == 0) for j in [decipher.a2i(t) for t in string])
        else 'M'
    )

def distance_calculator(start, end, to=True, as_numbers=False, print_str=True):
    distances = [
        (start, is_odd_or_even(start)),
        (end, is_odd_or_even(end)),
    ]
    completed = []
    pos = 0
    max_iterations = 1000
    max_count = 672
    k = 0
    halt = False
    while (pos < max_iterations) and not halt:
        for item in combinations([d[0] for d in distances], 2):
            if item in completed:
                continue
            c = item[0]
            s = item[1]
            for i in range(6):
                z = ''
                for j in range(len(c)):
                    z += distanceto(c[j], s[j]) if to else distancefrom(c[j], s[j])
                
                e = is_odd_or_even(z)
                if (z, e) not in distances:
                    distances.append((z, e))
                    p = ' '.join([str(decipher.a2i(t)).zfill(2) for t in z])
                    if print_str:
                        print('{} ({}, {}) = {} {}'.format(str(k).zfill(2), c, s, (p if as_numbers else z), e))
                    k += 1
                c = s
                s = z

            completed.append(item)
            
            # The cipher never exceeds k == 672.
            # Either at this point there are no new values or the number
            # of combinations being searched is simply too large to
            # complete in a reasonable amount of time with a single processor
            if k >= max_count:
                halt=True
                break
                
        pos += 1

    return distances

Set of prior determined wheel mappings referenced here as a copy location

```
00 QQPRNGKSSNYPVTTMZFPK UTRNDNNNMJVRRWRJNEGD DCBVPGCUTVWBVCXWNYQS
01 UTRNDNNNMJVRRWRJNEGD DCBVPGCUTVWBVCXWNYQS IIJHLSOGGLAJDFFMZTJO
02 DCBVPGCUTVWBVCXWNYQS IIJHLSOGGLAJDFFMZTJO EFHLVLLLMPDHHCHPLUSV
03 IIJHLSOGGLAJDFFMZTJO EFHLVLLLMPDHHCHPLUSV VWXDJSWEFDCXDWBCLAIG
04 EFHLVLLLMPDHHCHPLUSV VWXDJSWEFDCXDWBCLAIG QQPRNGKSSNYPVTTMZFPK
05 VWXDJSWEFDCXDWBCLAIG QQPRNGKSSNYPVTTMZFPK UTRNDNNNMJVRRWRJNEGD
```

What I want to look at here is for each alternating distance, what wheel positions are found inside?

For example, what is between `QQPRNGKSSNYPVTTMZFPK` and `IIJHLSOGGLAJDFFMZTJO`

In [40]:
distance_calculator('QQPRNGKSSNYPVTTMZFPK', 'IIJHLSOGGLAJDFFMZTJO')

00 (QQPRNGKSSNYPVTTMZFPK, IIJHLSOGGLAJDFFMZTJO) = RRTPXLDNNXBTHLLZZNTD E
01 (IIJHLSOGGLAJDFFMZTJO, QQPRNGKSSNYPVTTMZFPK) = HHFJBNVLLBXFRNNZZLFV E
02 (QQPRNGKSSNYPVTTMZFPK, RRTPXLDNNXBTHLLZZNTD) = AADXJESUUJCDLRRMZHDS M
03 (IIJHLSOGGLAJDFFMZTJO, HHFJBNVLLBXFRNNZZLFV) = YYVBPUGEEPWVNHHMZRVG M
04 (RRTPXLDNNXBTHLLZZNTD, HHFJBNVLLBXFRNNZZLFV) = PPLTDBRXXDVLJBBZZXLR E
05 (HHFJBNVLLBXFRNNZZLFV, RRTPXLDNNXBTHLLZZNTD) = JJNFVXHBBVDNPXXZZBNH E
06 (QQPRNGKSSNYPVTTMZFPK, JJNFVXHBBVDNPXXZZBNH) = SSXNHQWIIHEXTDDMZVXW M
07 (IIJHLSOGGLAJDFFMZTJO, PPLTDBRXXDVLJBBZZXLR) = GGBLRICQQRUBFVVMZDBC M
08 (RRTPXLDNNXBTHLLZZNTD, PPLTDBRXXDVLJBBZZXLR) = XXRDFPNJJFTRBPPZZJRN E
09 (HHFJBNVLLBXFRNNZZLFV, JJNFVXHBBVDNPXXZZBNH) = BBHVTJLPPTFHXJJZZPHL E
10 (AADXJESUUJCDLRRMZHDS, PPLTDBRXXDVLJBBZZXLR) = OOHVTWYCCTSHXJJMZPHY M
11 (YYVBPUGEEPWVNHHMZRVG, JJNFVXHBBVDNPXXZZBNH) = KKRDFCAWWFGRBPPMZJRA M
12 (PPLTDBRXXDVLJBBZZXLR, JJNFVXHBBVDNPXXZZBNH) = TTBLRVPDDRHBFVVZZDBP E
13 (JJNFVXHBBVDNPXXZZBNH, PPLTDBRXXDVLJBBZZXLR) = F

[('QQPRNGKSSNYPVTTMZFPK', 'M'),
 ('IIJHLSOGGLAJDFFMZTJO', 'M'),
 ('RRTPXLDNNXBTHLLZZNTD', 'E'),
 ('HHFJBNVLLBXFRNNZZLFV', 'E'),
 ('AADXJESUUJCDLRRMZHDS', 'M'),
 ('YYVBPUGEEPWVNHHMZRVG', 'M'),
 ('PPLTDBRXXDVLJBBZZXLR', 'E'),
 ('JJNFVXHBBVDNPXXZZBNH', 'E'),
 ('SSXNHQWIIHEXTDDMZVXW', 'M'),
 ('GGBLRICQQRUBFVVMZDBC', 'M'),
 ('XXRDFPNJJFTRBPPZZJRN', 'E'),
 ('BBHVTJLPPTFHXJJZZPHL', 'E'),
 ('OOHVTWYCCTSHXJJMZPHY', 'M'),
 ('KKRDFCAWWFGRBPPMZJRA', 'M'),
 ('TTBLRVPDDRHBFVVZZDBP', 'E'),
 ('FFXNHDJVVHRXTDDZZVXJ', 'E'),
 ('CCLTDOEKKDILJBBMZXLE', 'M'),
 ('WWNFVKUOOVQNPXXMZBNU', 'M'),
 ('NNDXJRFHHJPDLRRZZHDF', 'E'),
 ('LLVBPHTRRPJVNHHZZRVT', 'E'),
 ('EETPXYQAAXOTHLLMZNTQ', 'M'),
 ('UUFJBAIYYBKFRNNMZLFI', 'M'),
 ('DDPRNTXFFNLPVTTZZFPX', 'E'),
 ('VVJHLFBTTLNJDFFZZTJB', 'E'),
 ('MMZZZMMMMZMZZZZMZZZM', 'M')]

In [41]:
distance_calculator('QQPRNGKSSNYPVTTMZFPK', 'IIJHLSOGGLAJDFFMZTJO', as_numbers=True)

00 (QQPRNGKSSNYPVTTMZFPK, IIJHLSOGGLAJDFFMZTJO) = 18 18 20 16 24 12 04 14 14 24 02 20 08 12 12 26 26 14 20 04 E
01 (IIJHLSOGGLAJDFFMZTJO, QQPRNGKSSNYPVTTMZFPK) = 08 08 06 10 02 14 22 12 12 02 24 06 18 14 14 26 26 12 06 22 E
02 (QQPRNGKSSNYPVTTMZFPK, RRTPXLDNNXBTHLLZZNTD) = 01 01 04 24 10 05 19 21 21 10 03 04 12 18 18 13 26 08 04 19 M
03 (IIJHLSOGGLAJDFFMZTJO, HHFJBNVLLBXFRNNZZLFV) = 25 25 22 02 16 21 07 05 05 16 23 22 14 08 08 13 26 18 22 07 M
04 (RRTPXLDNNXBTHLLZZNTD, HHFJBNVLLBXFRNNZZLFV) = 16 16 12 20 04 02 18 24 24 04 22 12 10 02 02 26 26 24 12 18 E
05 (HHFJBNVLLBXFRNNZZLFV, RRTPXLDNNXBTHLLZZNTD) = 10 10 14 06 22 24 08 02 02 22 04 14 16 24 24 26 26 02 14 08 E
06 (QQPRNGKSSNYPVTTMZFPK, JJNFVXHBBVDNPXXZZBNH) = 19 19 24 14 08 17 23 09 09 08 05 24 20 04 04 13 26 22 24 23 M
07 (IIJHLSOGGLAJDFFMZTJO, PPLTDBRXXDVLJBBZZXLR) = 07 07 02 12 18 09 03 17 17 18 21 02 06 22 22 13 26 04 02 03 M
08 (RRTPXLDNNXBTHLLZZNTD, PPLTDBRXXDVLJBBZZXLR) = 24 24 18 04 06 16 14 10 10 06 20 18 02 16 16 26 26 10 

[('QQPRNGKSSNYPVTTMZFPK', 'M'),
 ('IIJHLSOGGLAJDFFMZTJO', 'M'),
 ('RRTPXLDNNXBTHLLZZNTD', 'E'),
 ('HHFJBNVLLBXFRNNZZLFV', 'E'),
 ('AADXJESUUJCDLRRMZHDS', 'M'),
 ('YYVBPUGEEPWVNHHMZRVG', 'M'),
 ('PPLTDBRXXDVLJBBZZXLR', 'E'),
 ('JJNFVXHBBVDNPXXZZBNH', 'E'),
 ('SSXNHQWIIHEXTDDMZVXW', 'M'),
 ('GGBLRICQQRUBFVVMZDBC', 'M'),
 ('XXRDFPNJJFTRBPPZZJRN', 'E'),
 ('BBHVTJLPPTFHXJJZZPHL', 'E'),
 ('OOHVTWYCCTSHXJJMZPHY', 'M'),
 ('KKRDFCAWWFGRBPPMZJRA', 'M'),
 ('TTBLRVPDDRHBFVVZZDBP', 'E'),
 ('FFXNHDJVVHRXTDDZZVXJ', 'E'),
 ('CCLTDOEKKDILJBBMZXLE', 'M'),
 ('WWNFVKUOOVQNPXXMZBNU', 'M'),
 ('NNDXJRFHHJPDLRRZZHDF', 'E'),
 ('LLVBPHTRRPJVNHHZZRVT', 'E'),
 ('EETPXYQAAXOTHLLMZNTQ', 'M'),
 ('UUFJBAIYYBKFRNNMZLFI', 'M'),
 ('DDPRNTXFFNLPVTTZZFPX', 'E'),
 ('VVJHLFBTTLNJDFFZZTJB', 'E'),
 ('MMZZZMMMMZMZZZZMZZZM', 'M')]

So this is interesting. Most of these, in effect look like ciphers of their own, and potentially they are... However, there is one at position 22 which really stands out.

As we look at the pattern `MMZZZMMMMZMZZZZMZZZM`, we notice that it is composed entirely of `M` and `Z` but this isn't random.

Take a look at it inline with the original ciphertext:

```
QQPRNGKSSNYPVTTMZFPK
MMZZZMMMMZMZZZZMZZZM
```

The letter Q is the 17th letter of the alphabet. This is written as `M`.
`P`, `R` and `N` on the other hand, are all even numbered. These are written as the letter `Z`.

This is a binary system. M for odd, Z for Even.

Question: How does the pattern change if we look at the next pair in the list?

In [42]:
distance_calculator('DCBVPGCUTVWBVCXWNYQS', 'VWXDJSWEFDCXDWBCLAIG')

00 (DCBVPGCUTVWBVCXWNYQS, VWXDJSWEFDCXDWBCLAIG) = RTVHTLTJLHFVHTDFXBRN E
01 (VWXDJSWEFDCXDWBCLAIG, DCBVPGCUTVWBVCXWNYQS) = HFDRFNFPNRTDRFVTBXHL E
02 (DCBVPGCUTVWBVCXWNYQS, RTVHTLTJLHFVHTDFXBRN) = NQTLDEQORLITLQFIJCAU M
03 (VWXDJSWEFDCXDWBCLAIG, HFDRFNFPNRTDRFVTBXHL) = LIFNVUIKHNQFNITQPWYE M
04 (RTVHTLTJLHFVHTDFXBRN, HFDRFNFPNRTDRFVTBXHL) = PLHJLBLFBJNHJLRNDVPX E
05 (HFDRFNFPNRTDRFVTBXHL, RTVHTLTJLHFVHTDFXBRN) = JNRPNXNTXPLRPNHLVDJB E
06 (DCBVPGCUTVWBVCXWNYQS, JNRPNXNTXPLRPNHLVDJB) = FKPTXQKYDTOPTKJOHESI M
07 (VWXDJSWEFDCXDWBCLAIG, PLHJLBLFBJNHJLRNDVPX) = TOJFBIOAVFKJFOPKRUGQ M
08 (RTVHTLTJLHFVHTDFXBRN, PLHJLBLFBJNHJLRNDVPX) = XRLBRPRVPBHLBRNHFTXJ E
09 (HFDRFNFPNRTDRFVTBXHL, JNRPNXNTXPLRPNHLVDJB) = BHNXHJHDJXRNXHLRTFBP E
10 (NQTLDEQORLITLQFIJCAU, PLHJLBLFBJNHJLRNDVPX) = BUNXHWUQJXENXULETSOC M
11 (LIFNVUIKHNQFNITQPWYE, JNRPNXNTXPLRPNHLVDJB) = XELBRCEIPBULBENUFGKW M
12 (PLHJLBLFBJNHJLRNDVPX, JNRPNXNTXPLRPNHLVDJB) = TBJFBVBNVFXJFBPXRHTD E
13 (JNRPNXNTXPLRPNHLVDJB, PLHJLBLFBJNHJLRNDVPX) = F

[('DCBVPGCUTVWBVCXWNYQS', 'M'),
 ('VWXDJSWEFDCXDWBCLAIG', 'M'),
 ('RTVHTLTJLHFVHTDFXBRN', 'E'),
 ('HFDRFNFPNRTDRFVTBXHL', 'E'),
 ('NQTLDEQORLITLQFIJCAU', 'M'),
 ('LIFNVUIKHNQFNITQPWYE', 'M'),
 ('PLHJLBLFBJNHJLRNDVPX', 'E'),
 ('JNRPNXNTXPLRPNHLVDJB', 'E'),
 ('FKPTXQKYDTOPTKJOHESI', 'M'),
 ('TOJFBIOAVFKJFOPKRUGQ', 'M'),
 ('XRLBRPRVPBHLBRNHFTXJ', 'E'),
 ('BHNXHJHDJXRNXHLRTFBP', 'E'),
 ('BUNXHWUQJXENXULETSOC', 'M'),
 ('XELBRCEIPBULBENUFGKW', 'M'),
 ('TBJFBVBNVFXJFBPXRHTD', 'E'),
 ('FXPTXDXLDTBPTXJBHRFV', 'E'),
 ('PYHJLOYSBJAHJYRADICK', 'M'),
 ('JARPNKAGXPYRPAHYVQWO', 'M'),
 ('NDTLDRDBRLVTLDFVJPNH', 'E'),
 ('LVFNVHVXHNDFNVTDPJLR', 'E'),
 ('RGVHTYGWLHSVHGDSXOEA', 'M'),
 ('HSDRFASCNRGDRSVGBKUY', 'M'),
 ('DPBVPTPHTVJBVPXJNLDF', 'E'),
 ('VJXDJFJRFDPXDJBPLNVT', 'E'),
 ('ZMZZZMMMZZMZZMZMZMMM', 'M')]

In [43]:
distance_calculator('DCBVPGCUTVWBVCXWNYQS', 'VWXDJSWEFDCXDWBCLAIG', as_numbers=True)

00 (DCBVPGCUTVWBVCXWNYQS, VWXDJSWEFDCXDWBCLAIG) = 18 20 22 08 20 12 20 10 12 08 06 22 08 20 04 06 24 02 18 14 E
01 (VWXDJSWEFDCXDWBCLAIG, DCBVPGCUTVWBVCXWNYQS) = 08 06 04 18 06 14 06 16 14 18 20 04 18 06 22 20 02 24 08 12 E
02 (DCBVPGCUTVWBVCXWNYQS, RTVHTLTJLHFVHTDFXBRN) = 14 17 20 12 04 05 17 15 18 12 09 20 12 17 06 09 10 03 01 21 M
03 (VWXDJSWEFDCXDWBCLAIG, HFDRFNFPNRTDRFVTBXHL) = 12 09 06 14 22 21 09 11 08 14 17 06 14 09 20 17 16 23 25 05 M
04 (RTVHTLTJLHFVHTDFXBRN, HFDRFNFPNRTDRFVTBXHL) = 16 12 08 10 12 02 12 06 02 10 14 08 10 12 18 14 04 22 16 24 E
05 (HFDRFNFPNRTDRFVTBXHL, RTVHTLTJLHFVHTDFXBRN) = 10 14 18 16 14 24 14 20 24 16 12 18 16 14 08 12 22 04 10 02 E
06 (DCBVPGCUTVWBVCXWNYQS, JNRPNXNTXPLRPNHLVDJB) = 06 11 16 20 24 17 11 25 04 20 15 16 20 11 10 15 08 05 19 09 M
07 (VWXDJSWEFDCXDWBCLAIG, PLHJLBLFBJNHJLRNDVPX) = 20 15 10 06 02 09 15 01 22 06 11 10 06 15 16 11 18 21 07 17 M
08 (RTVHTLTJLHFVHTDFXBRN, PLHJLBLFBJNHJLRNDVPX) = 24 18 12 02 18 16 18 22 16 02 08 12 02 18 14 08 06 20 

[('DCBVPGCUTVWBVCXWNYQS', 'M'),
 ('VWXDJSWEFDCXDWBCLAIG', 'M'),
 ('RTVHTLTJLHFVHTDFXBRN', 'E'),
 ('HFDRFNFPNRTDRFVTBXHL', 'E'),
 ('NQTLDEQORLITLQFIJCAU', 'M'),
 ('LIFNVUIKHNQFNITQPWYE', 'M'),
 ('PLHJLBLFBJNHJLRNDVPX', 'E'),
 ('JNRPNXNTXPLRPNHLVDJB', 'E'),
 ('FKPTXQKYDTOPTKJOHESI', 'M'),
 ('TOJFBIOAVFKJFOPKRUGQ', 'M'),
 ('XRLBRPRVPBHLBRNHFTXJ', 'E'),
 ('BHNXHJHDJXRNXHLRTFBP', 'E'),
 ('BUNXHWUQJXENXULETSOC', 'M'),
 ('XELBRCEIPBULBENUFGKW', 'M'),
 ('TBJFBVBNVFXJFBPXRHTD', 'E'),
 ('FXPTXDXLDTBPTXJBHRFV', 'E'),
 ('PYHJLOYSBJAHJYRADICK', 'M'),
 ('JARPNKAGXPYRPAHYVQWO', 'M'),
 ('NDTLDRDBRLVTLDFVJPNH', 'E'),
 ('LVFNVHVXHNDFNVTDPJLR', 'E'),
 ('RGVHTYGWLHSVHGDSXOEA', 'M'),
 ('HSDRFASCNRGDRSVGBKUY', 'M'),
 ('DPBVPTPHTVJBVPXJNLDF', 'E'),
 ('VJXDJFJRFDPXDJBPLNVT', 'E'),
 ('ZMZZZMMMZZMZZMZMZMMM', 'M')]

And the final pair?

In [44]:
distance_calculator('UTRNDNNNMJVRRWRJNEGD', 'EFHLVLLLMPDHHCHPLUSV')

00 (UTRNDNNNMJVRRWRJNEGD, EFHLVLLLMPDHHCHPLUSV) = JLPXRXXXZFHPPFPFXPLR E
01 (EFHLVLLLMPDHHCHPLUSV, UTRNDNNNMJVRRWRJNEGD) = PNJBHBBBZTRJJTJTBJNH E
02 (UTRNDNNNMJVRRWRJNEGD, JLPXRXXXZFHPPFPFXPLR) = ORXJNJJJMVLXXIXVJKEN M
03 (EFHLVLLLMPDHHCHPLUSV, PNJBHBBBZTRJJTJTBJNH) = KHBPLPPPMDNBBQBDPOUL M
04 (JLPXRXXXZFHPPFPFXPLR, PNJBHBBBZTRJJTJTBJNH) = FBTDPDDDZNJTTNTNDTBP E
05 (PNJBHBBBZTRJJTJTBJNH, JLPXRXXXZFHPPFPFXPLR) = TXFVJVVVZLPFFLFLVFXJ E
06 (UTRNDNNNMJVRRWRJNEGD, TXFVJVVVZLPFFLFLVFXJ) = YDNHFHHHMBTNNONBHAQF M
07 (EFHLVLLLMPDHHCHPLUSV, FBTDPDDDZNJTTNTNDTBP) = AVLRTRRRMXFLLKLXRYIT M
08 (JLPXRXXXZFHPPFPFXPLR, FBTDPDDDZNJTTNTNDTBP) = VPDFXFFFZHBDDHDHFDPX E
09 (PNJBHBBBZTRJJTJTBJNH, TXFVJVVVZLPFFLFLVFXJ) = DJVTBTTTZRXVVRVRTVJB E
10 (ORXJNJJJMVLXXIXVJKEN, FBTDPDDDZNJTTNTNDTBP) = QJVTBTTTMRXVVEVRTIWB M
11 (KHBPLPPPMDNBBQBDPOUL, TXFVJVVVZLPFFLFLVFXJ) = IPDFXFFFMHBDDUDHFQCX M
12 (FBTDPDDDZNJTTNTNDTBP, TXFVJVVVZLPFFLFLVFXJ) = NVLRTRRRZXFLLXLXRLVT E
13 (TXFVJVVVZLPFFLFLVFXJ, FBTDPDDDZNJTTNTNDTBP) = L

[('UTRNDNNNMJVRRWRJNEGD', 'M'),
 ('EFHLVLLLMPDHHCHPLUSV', 'M'),
 ('JLPXRXXXZFHPPFPFXPLR', 'E'),
 ('PNJBHBBBZTRJJTJTBJNH', 'E'),
 ('ORXJNJJJMVLXXIXVJKEN', 'M'),
 ('KHBPLPPPMDNBBQBDPOUL', 'M'),
 ('FBTDPDDDZNJTTNTNDTBP', 'E'),
 ('TXFVJVVVZLPFFLFLVFXJ', 'E'),
 ('YDNHFHHHMBTNNONBHAQF', 'M'),
 ('AVLRTRRRMXFLLKLXRYIT', 'M'),
 ('VPDFXFFFZHBDDHDHFDPX', 'E'),
 ('DJVTBTTTZRXVVRVRTVJB', 'E'),
 ('QJVTBTTTMRXVVEVRTIWB', 'M'),
 ('IPDFXFFFMHBDDUDHFQCX', 'M'),
 ('NVLRTRRRZXFLLXLXRLVT', 'E'),
 ('LDNHFHHHZBTNNBNBHNDF', 'E'),
 ('SBTDPDDDMNJTTATNDGOP', 'M'),
 ('GXFVJVVVMLPFFYFLVSKJ', 'M'),
 ('BRXJNJJJZVLXXVXVJXRN', 'E'),
 ('XHBPLPPPZDNBBDBDPBHL', 'E'),
 ('WLPXRXXXMFHPPSPFXCYR', 'M'),
 ('CNJBHBBBMTRJJGJTBWAH', 'M'),
 ('HTRNDNNNZJVRRJRJNRTD', 'E'),
 ('RFHLVLLLZPDHHPHPLHFV', 'E'),
 ('MZZZZZZZMZZZZMZZZMMZ', 'M')]

In [45]:
distance_calculator('UTRNDNNNMJVRRWRJNEGD', 'EFHLVLLLMPDHHCHPLUSV', as_numbers=True)

00 (UTRNDNNNMJVRRWRJNEGD, EFHLVLLLMPDHHCHPLUSV) = 10 12 16 24 18 24 24 24 26 06 08 16 16 06 16 06 24 16 12 18 E
01 (EFHLVLLLMPDHHCHPLUSV, UTRNDNNNMJVRRWRJNEGD) = 16 14 10 02 08 02 02 02 26 20 18 10 10 20 10 20 02 10 14 08 E
02 (UTRNDNNNMJVRRWRJNEGD, JLPXRXXXZFHPPFPFXPLR) = 15 18 24 10 14 10 10 10 13 22 12 24 24 09 24 22 10 11 05 14 M
03 (EFHLVLLLMPDHHCHPLUSV, PNJBHBBBZTRJJTJTBJNH) = 11 08 02 16 12 16 16 16 13 04 14 02 02 17 02 04 16 15 21 12 M
04 (JLPXRXXXZFHPPFPFXPLR, PNJBHBBBZTRJJTJTBJNH) = 06 02 20 04 16 04 04 04 26 14 10 20 20 14 20 14 04 20 02 16 E
05 (PNJBHBBBZTRJJTJTBJNH, JLPXRXXXZFHPPFPFXPLR) = 20 24 06 22 10 22 22 22 26 12 16 06 06 12 06 12 22 06 24 10 E
06 (UTRNDNNNMJVRRWRJNEGD, TXFVJVVVZLPFFLFLVFXJ) = 25 04 14 08 06 08 08 08 13 02 20 14 14 15 14 02 08 01 17 06 M
07 (EFHLVLLLMPDHHCHPLUSV, FBTDPDDDZNJTTNTNDTBP) = 01 22 12 18 20 18 18 18 13 24 06 12 12 11 12 24 18 25 09 20 M
08 (JLPXRXXXZFHPPFPFXPLR, FBTDPDDDZNJTTNTNDTBP) = 22 16 04 06 24 06 06 06 26 08 02 04 04 08 04 08 06 04 

[('UTRNDNNNMJVRRWRJNEGD', 'M'),
 ('EFHLVLLLMPDHHCHPLUSV', 'M'),
 ('JLPXRXXXZFHPPFPFXPLR', 'E'),
 ('PNJBHBBBZTRJJTJTBJNH', 'E'),
 ('ORXJNJJJMVLXXIXVJKEN', 'M'),
 ('KHBPLPPPMDNBBQBDPOUL', 'M'),
 ('FBTDPDDDZNJTTNTNDTBP', 'E'),
 ('TXFVJVVVZLPFFLFLVFXJ', 'E'),
 ('YDNHFHHHMBTNNONBHAQF', 'M'),
 ('AVLRTRRRMXFLLKLXRYIT', 'M'),
 ('VPDFXFFFZHBDDHDHFDPX', 'E'),
 ('DJVTBTTTZRXVVRVRTVJB', 'E'),
 ('QJVTBTTTMRXVVEVRTIWB', 'M'),
 ('IPDFXFFFMHBDDUDHFQCX', 'M'),
 ('NVLRTRRRZXFLLXLXRLVT', 'E'),
 ('LDNHFHHHZBTNNBNBHNDF', 'E'),
 ('SBTDPDDDMNJTTATNDGOP', 'M'),
 ('GXFVJVVVMLPFFYFLVSKJ', 'M'),
 ('BRXJNJJJZVLXXVXVJXRN', 'E'),
 ('XHBPLPPPZDNBBDBDPBHL', 'E'),
 ('WLPXRXXXMFHPPSPFXCYR', 'M'),
 ('CNJBHBBBMTRJJGJTBWAH', 'M'),
 ('HTRNDNNNZJVRRJRJNRTD', 'E'),
 ('RFHLVLLLZPDHHPHPLHFV', 'E'),
 ('MZZZZZZZMZZZZMZZZMMZ', 'M')]

The part that I find fascinating here is this only occurs on strings that are `Z` mapped to each other.

This means that you cannot look across pairs to find a new pattern. Instead, looking across pairs, for example `QQPRNGKSSNYPVTTMZFPK` and `UTRNDNNNMJVRRWRJNEGD`, does not give a new pattern, but will contain all 3 patterns.

- `MMZZZMMMMZMZZZZMZZZM` - row 488
- `MZZZZZZZMZZZZMZZZMMZ` - row 515
- `ZMZZZMMMZZMZZMZMZMMM` - row 540

In [46]:
_ = distance_calculator('QQPRNGKSSNYPVTTMZFPK', 'UTRNDNNNMJVRRWRJNEGD', True)

00 (QQPRNGKSSNYPVTTMZFPK, UTRNDNNNMJVRRWRJNEGD) = DCBVPGCUTVWBVCXWNYQS M
01 (UTRNDNNNMJVRRWRJNEGD, DCBVPGCUTVWBVCXWNYQS) = IIJHLSOGGLAJDFFMZTJO M
02 (DCBVPGCUTVWBVCXWNYQS, IIJHLSOGGLAJDFFMZTJO) = EFHLVLLLMPDHHCHPLUSV M
03 (IIJHLSOGGLAJDFFMZTJO, EFHLVLLLMPDHHCHPLUSV) = VWXDJSWEFDCXDWBCLAIG M
04 (QQPRNGKSSNYPVTTMZFPK, DCBVPGCUTVWBVCXWNYQS) = MLLDBZRBAHXLZIDJNSAH M
05 (IIJHLSOGGLAJDFFMZTJO, VWXDJSWEFDCXDWBCLAIG) = MNNVXZHXYRBNZQVPLGYR M
06 (QQPRNGKSSNYPVTTMZFPK, IIJHLSOGGLAJDFFMZTJO) = RRTPXLDNNXBTHLLZZNTD E
07 (IIJHLSOGGLAJDFFMZTJO, QQPRNGKSSNYPVTTMZFPK) = HHFJBNVLLBXFRNNZZLFV E
08 (QQPRNGKSSNYPVTTMZFPK, EFHLVLLLMPDHHCHPLUSV) = NORTHEASTBERLINCLOCK M
09 (IIJHLSOGGLAJDFFMZTJO, UTRNDNNNMJVRRWRJNEGD) = LKHFRUYGFXUHNQLWNKWO M
10 (UTRNDNNNMJVRRWRJNEGD, EFHLVLLLMPDHHCHPLUSV) = JLPXRXXXZFHPPFPFXPLR E
11 (EFHLVLLLMPDHHCHPLUSV, UTRNDNNNMJVRRWRJNEGD) = PNJBHBBBZTRJJTJTBJNH E
12 (UTRNDNNNMJVRRWRJNEGD, VWXDJSWEFDCXDWBCLAIG) = ACFPFEIQSTGFLZJSXVBC M
13 (EFHLVLLLMPDHHCHPLUSV, DCBVPGCUTVWBVCXWNYQS) = Y

124 (VUVLNSGIHTYVDOJWNMKW, ZBBRVZPVXJDBZHRFXNXJ) = DGFFHGIMPPEFVSHIJAMM M
125 (DEDNLGSQRFADVKPCLMOC, ZXXHDZJDBPVXZRHTBLBP) = VSTTRSQMJJUTDGRQPYMM M
126 (VUVLNSGIHTYVDOJWNMKW, UVTFZNDJKTZTREJPLSEN) = YAXTLUWACZAXNPZSXFTQ M
127 (DEDNLGSQRFADVKPCLMOC, EDFTZLVPOFZFHUPJNGUL) = AYBFNECYWZYBLJZGBTFI M
128 (VUVLNSGIHTYVDOJWNMKW, EHJDRLBHKZHJHKZVJIQF) = IMNRDSUYCFINDVPYVVFI M
129 (DEDNLGSQRFADVKPCLMOC, URPVHNXROZRPROZDPQIT) = QMLHVGEAWTQLVDJADDTQ M
130 (VUVLNSGIHTYVDOJWNMKW, PPLTDBRXXDVLJBBZZXLR) = TUPHPIKOPJWPFMRCLKAU M
131 (DEDNLGSQRFADVKPCLMOC, JJNFVXHBBVDNPXXZZBNH) = FEJRJQOKJPCJTMHWNOYE M
132 (VUVLNSGIHTYVDOJWNMKW, JNRPNXNTXPLRPNHLVDJB) = NSVDZEGKPVMVLYXOHQYE M
133 (DEDNLGSQRFADVKPCLMOC, PLHJLBLFBJNHJLRNDVPX) = LGDVZUSOJDMDNABKRIAU M
134 (AADXJESUUJCDLRRMZHDS, PPLTDBRXXDVLJBBZZXLR) = OOHVTWYCCTSHXJJMZPHY M
135 (YYVBPUGEEPWVNHHMZRVG, JJNFVXHBBVDNPXXZZBNH) = KKRDFCAWWFGRBPPMZJRA M
136 (AADXJESUUJCDLRRMZHDS, TSNPTIUSRZSNFEZWNWCK) = SRJRJDBXWPPJTMHJNOYR M
137 (YYVBPUGEEPWVNHHMZRVG, FGLJFQEGHZG

266 (DCBVPGCUTVWBVCXWNYQS, TBJFBVBNVFXJFBPXRHTD) = PYHJLOYSBJAHJYRADICK M
267 (VWXDJSWEFDCXDWBCLAIG, FXPTXDXLDTBPTXJBHRFV) = JARPNKAGXPYRPAHYVQWO M
268 (MLLDBZRBAHXLZIDJNSAH, ZDDJRZFRVTHDZPJLVBVT) = MRRFPZNPULJRZGFBHIUL M
269 (MNNVXZHXYRBNZQVPLGYR, ZVVPHZTHDFRVZJPNDXDF) = MHHTJZLJENPHZSTXRQEN M
270 (MLLDBZRBAHXLZIDJNSAH, UXVXVNTFIDDVRMBVJGCX) = HLJTTNBDHVFJRDXLVNBP E
271 (MNNVXZHXYRBNZQVPLGYR, EBDBDLFTQVVDHMXDPSWB) = RNPFFLXVRDTPHVBNDLXJ E
272 (MLLDBZRBAHXLZIDJNSAH, EJLVNLRDIJLLHSRBHWOP) = RXZRLLZBHBNZHJNRTDNH E
273 (MNNVXZHXYRBNZQVPLGYR, UPNDLNHVQPNNRGHXRCKJ) = HBZHNNZXRXLZRPLHFVLR E
274 (MLLDBZRBAHXLZIDJNSAH, PRNLZBHTVNZNJJTFXLJB) = CFBHXBPRUFBBJAPVJSIT M
275 (MNNVXZHXYRBNZQVPLGYR, JHLNZXRFDLZLPPFTBNPX) = WTXRBXJHETXXPYJDPGQF M
276 (MLLDBZRBAHXLZIDJNSAH, JPTHJXDPVZPTPVZRTRHL) = WDHDHXLNURRHPMVHFYGD M
277 (MNNVXZHXYRBNZQVPLGYR, PJFRPBVJDZJFJDZHFHRN) = CVRVRBNLEHHRJMDRTASV M
278 (MLLDBZRBAHXLZIDJNSAH, KLFZDPVHIXVFBGLPLQQF) = XZTVBPDFHPXTBXHFXXPX E
279 (MNNVXZHXYRBNZQVPLGYR, ONTZVJDRQBD

388 (FINBBQUCFJKNTCRIJQUY, FXPTXDXLDTBPTXJBHRFV) = ZOBRVMCIXJQBZURSXAKW M
389 (TQLXXIEWTPOLFWHQPIEA, TBJFBVBNVFXJFBPXRHTD) = ZKXHDMWQBPIXZEHGBYOC M
390 (WZDTPXFVYXJDPWLVJWKJ, FXPTXDXLDTBPTXJBHRFV) = IXLZHFRPEVRLDAXFXUUL M
391 (CZVFJBTDABPVJCNDPCOP, TBJFBVBNVFXJFBPXRHTD) = QBNZRTHJUDHNVYBTBEEN M
392 (NQTLDEQORLITLQFIJCAU, FXPTXDXLDTBPTXJBHRFV) = RGVHTYGWLHSVHGDSXOEA M
393 (LIFNVUIKHNQFNITQPWYE, TBJFBVBNVFXJFBPXRHTD) = HSDRFASCNRGDRSVGBKUY M
394 (ZBBRVZPVXJDBZHRFXNXJ, ZVVPHZTHDFRVZJPNDXDF) = ZTTXLZDLFVNTZBXHFJFV E
395 (ZXXHDZJDBPVXZRHTBLBP, ZDDJRZFRVTHDZPJLVBVT) = ZFFBNZVNTDLFZXBRTPTD E
396 (ZBBRVZPVXJDBZHRFXNXJ, EBDBDLFTQVVDHMXDPSWB) = EZBJHLPXSLRBHEFXREYR M
397 (ZXXHDZJDBPVXZRHTBLBP, UXVXVNTFIDDVRMBVJGCX) = UZXPRNJBGNHXRUTBHUAH M
398 (ZBBRVZPVXJDBZHRFXNXJ, UPNDLNHVQPNNRGHXRCKJ) = UNLLPNRZSFJLRYPRTOMZ M
399 (ZXXHDZJDBPVXZRHTBLBP, EJLVNLRDIJLLHSRBHWOP) = ELNNJLHZGTPNHAJHFKMZ M
400 (ZBBRVZPVXJDBZHRFXNXJ, JHLNZXRFDLZLPPFTBNPX) = JFJVDXBJFBVJPHNNDZRN E
401 (ZXXHDZJDBPVXZRHTBLBP, PRNLZBHTVNZ

505 (OPVRRJTNOLHVXAFPLWGD, LDNHFHHHZBTNNBNBHNDF) = WNRPNXNTKPLRPAHLVQWB M
506 (KJDHHPFLKNRDBYTJNCSV, NVLRTRRRZXFLLXLXRLVT) = CLHJLBLFOJNHJYRNDICX M
507 (KMTVBCQSUPKTBXHSXXPK, LDNHFHHHZBTNNBNBHNDF) = AQTLDEQOELITLDFIJPNU M
508 (OMFDXWIGEJOFXBRGBBJO, NVLRTRRRZXFLLXLXRLVT) = YIFNVUIKUNQFNVTQPJLE M
509 (GJRZLVNXATNRFUJVJYYR, LDNHFHHHZBTNNBNBHNDF) = ETVHTLTJYHFVHGDFXOEN M
510 (SPHZNDLBYFLHTEPDPAAH, NVLRTRRRZXFLLXLXRLVT) = UFDRFNFPARTDRSVTBKUL M
511 (CGPDVOKCGXQPJRLYVZHY, LDNHFHHHZBTNNBNBHNDF) = IWXDJSWESDCXDJBCLNVG M
512 (WSJVDKOWSBIJPHNADZRA, NVLRTRRRZXFLLXLXRLVT) = QCBVPGCUGVWBVPXWNLDS M
513 (YURRXUAMIVORNRXADPZM, TBJFBVBNVFXJFBPXRHTD) = UGRNDAAAMJIRRJRWNRTQ M
514 (AEHHBEYMQDKHLHBYVJZM, FXPTXDXLDTBPTXJBHRFV) = ESHLVYYYMPQHHPHCLHFI M
515 (YDNHFHHHMBTNNONBHAQF, LDNHFHHHZBTNNBNBHNDF) = MZZZZZZZMZZZZMZZZMMZ M
516 (YDNHFHHHMBTNNONBHAQF, UMXPRAWOGNUXRHTOHHNU) = VIJHLSOGTLAJDSFMZGWO M
517 (AVLRTRRRMXFLLKLXRYIT, EMBJHYCKSLEBHRFKRRLE) = DQPRNGKSFNYPVGTMZSCK M
518 (YDNHFHHHMBTNNONBHAQF, DVHXDTLVNZV

617 (OOHVTWYCCTSHXJJMZPHY, ZERFPMACHLWRZGFOHIUY) = KPJJVPBZERDJBWVBHSMZ M
618 (KKRDFCAWWFGRBPPMZJRA, ZUHTJMYWRNCHZSTKRQEA) = OJPPDJXZUHVPXCDXRGMZ M
619 (OOHVTWYCCTSHXJJMZPHY, VBPJZFXHNPZPDDHRTJDF) = GMHNFIYEKVGHFTXETTVG M
620 (KKRDFCAWWFGRBPPMZJRA, DXJPZTBRLJZJVVRHFPVT) = SMRLTQAUODSRTFBUFFDS M
621 (OOHVTWYCCTSHXJJMZPHY, RYNNJYUMTTCNHAJUFKMM) = CJFRPBVJQZJFJQZHFUEN M
622 (KKRDFCAWWFGRBPPMZJRA, HALLPAEMFFWLRYPETOMM) = WPTHJXDPIZPTPIZRTEUL M
623 (OOHVTWYCCTSHXJJMZPHY, NVLRTRRRZXFLLXLXRLVT) = YGDVZUSOWDMDNNBKRVNU M
624 (KKRDFCAWWFGRBPPMZJRA, LDNHFHHHZBTNNBNBHNDF) = ASVDZEGKCVMVLLXOHDLE M
625 (SRJRJDBXWPPJTMHJNOYR, NVLRTRRRZXFLLXLXRLVT) = UDBZJNPTCHPBRKDNDWWB M
626 (GHPHPVXBCJJPFMRPLKAH, LDNHFHHHZBTNNBNBHNDF) = EVXZPLJFWRJXHOVLVCCX M
627 (WULNZKESQLMLPPFGBNPK, NVLRTRRRZXFLLXLXRLVT) = QAZDTGMYILSZVHFQPXFI M
628 (CENLZOUGINMNJJTSXLJO, LDNHFHHHZBTNNBNBHNDF) = IYZVFSMAQNGZDRTIJBTQ M
629 (AXNJPRHNKHJNLSDDPMGD, NVLRTRRRZXFLLXLXRLVT) = MXXHDZJDOPVXZEHTBYOP M
630 (YBLPJHRLORPLNGVVJMSV, LDNHFHHHZBT

In [47]:
_ = distance_calculator('QQPRNGKSSNYPVTTMZFPK', 'DCBVPGCUTVWBVCXWNYQS', False)

00 (QQPRNGKSSNYPVTTMZFPK, DCBVPGCUTVWBVCXWNYQS) = EFHLVLLLMPDHHCHPLUSV M


Another interesting fact about this is that the patterns always appear at the same row numbers although the order of the pattern changes depending on which pairs are being examined.

Where you are looking at a given cipher and it's associated Z value, the pattern is always at position 22, whilst when you are looking across ciphers, the pattern is always at positions 488, 515 and 540.

Whether there is anything in this binary location or not is indeterminate but one thing this does seem to prove is there are two keys which unlock the first half of the cipher which can be recognised as follows:

- Key 1 XOR'd with ciphertext creates 
  - MZZZZZZZM at positions 26 - 34
  - ZZZZMZZZMMZ at positions 64 - 75
- Key 2 XOR'd with the ciphertext creates 
  - ZMZZZMMMZ at positions 26 - 34
  - ZMZZMZMZMMM at positions 64 - 75
  
Now we need two new methods, one XORs two values, returning M if either position is odd and Z if both positions are the same.

The second method will apply XOR to two entire strings, returning a string of M's and Z's

In [48]:
def xor(a, b):
    a = decipher.a2i(a) % 2 == 0
    b = decipher.a2i(b) % 2 == 0
    return 'Z' if a == b == 0 else 'M'

def xor_apply(cipher, intermediate):
    returnvalue = ''
    if len(cipher) != len(intermediate):
        print('Parameter lengths do not match')
        return None

    for i in range(len(cipher)):
        returnvalue += xor(cipher[i], intermediate[i])
    return returnvalue

def xor_verify(string):
    """
    Validates positions in the xor string to see if they match a known pattern
    """
    patterns = [
        ('MZZZZZZZM', 'ZZZZMZZZMMZ'),
        ('ZMZZZMMMZ', 'ZMZZMZMZMMM')
    ]
    flipped = [
        ('ZMMMMMMMZ', 'MMMMZMMMZZM'),
        ('MZMMMZZZM', 'MZMMZMZMZZZ')
    ]
    
    pos_a = string[25:33]
    pos_b = string[63:73]
    for pattern in patterns:
        if pos_a == pattern[0] and pos_b == pattern[1]:
            print('Possible match found')
            return True
        else:
            if pos_a == flipped[0] and pos_b == flipped[1]:
                print('Possible flipped match found')
                return False
    return False

There may also be the requirement to upgrade the wheels as the nature of these changes slightly.

We know that each position represented above is either odd, or even. That means that each wheel can turn only to the value it represents, e.g. An even wheel may only turn to an even position (B, D, F, H...) whilst an odd wheel may only turn to an odd position (A, C, E, G...)

So, revalation moment here. Last week I picked up on something whereby it's possible to split the alphabet into two halves by running the index position against the character and then mapping it to 'Z'

In the distance calculator function above, I have now included a flag to tell if the line is all odd, all even or mixed.

What is interesting about this flag is it tells me that many of the combinations end up all even regardless of the combination being used and none of them end up all odd.

Now, at face value, this might not tell us much but actually, it speaks volumes in hidden tones.

For example, if you look at the values for `distance_calculator('DCBVPGCUTVWBVCXWNYQS', 'VWXDJSWEFDCXDWBCLAIG')` above, there are 22 rows generated. 11 of these can be discounted immediately because they contain inverted pairs (pairs previously calculated but the other way around), this leaves 11 pairs. Of those, 6 are mixed leaving 5 unique even pairs.

Now look at the same for `distance_calculator('UTRNDNNNMJVRRWRJNEGD', 'EFHLVLLLMPDHHCHPLUSV')`, what we see is the exact same pattern.

Actually, I'm wrong, we don't discount the inverse patterns as they themselves are distinct, instead we use these to aid in targetting just the combinations we need.

## How?

OK so I guess this is the multi-million $ question: How do we target this?

I guess the first thing to do is work out if there is any pattern between the even spaced keys across keysets, so let's collect those:

In [49]:
distances = [
    [i for (i, e) in distance_calculator('QQPRNGKSSNYPVTTMZFPK', 'IIJHLSOGGLAJDFFMZTJO', print_str=False)],
    [i for (i, e) in distance_calculator('DCBVPGCUTVWBVCXWNYQS', 'VWXDJSWEFDCXDWBCLAIG', print_str=False)],
    [i for (i, e) in distance_calculator('UTRNDNNNMJVRRWRJNEGD', 'EFHLVLLLMPDHHCHPLUSV', print_str=False)],
]
distances

[['QQPRNGKSSNYPVTTMZFPK',
  'IIJHLSOGGLAJDFFMZTJO',
  'RRTPXLDNNXBTHLLZZNTD',
  'HHFJBNVLLBXFRNNZZLFV',
  'AADXJESUUJCDLRRMZHDS',
  'YYVBPUGEEPWVNHHMZRVG',
  'PPLTDBRXXDVLJBBZZXLR',
  'JJNFVXHBBVDNPXXZZBNH',
  'SSXNHQWIIHEXTDDMZVXW',
  'GGBLRICQQRUBFVVMZDBC',
  'XXRDFPNJJFTRBPPZZJRN',
  'BBHVTJLPPTFHXJJZZPHL',
  'OOHVTWYCCTSHXJJMZPHY',
  'KKRDFCAWWFGRBPPMZJRA',
  'TTBLRVPDDRHBFVVZZDBP',
  'FFXNHDJVVHRXTDDZZVXJ',
  'CCLTDOEKKDILJBBMZXLE',
  'WWNFVKUOOVQNPXXMZBNU',
  'NNDXJRFHHJPDLRRZZHDF',
  'LLVBPHTRRPJVNHHZZRVT',
  'EETPXYQAAXOTHLLMZNTQ',
  'UUFJBAIYYBKFRNNMZLFI',
  'DDPRNTXFFNLPVTTZZFPX',
  'VVJHLFBTTLNJDFFZZTJB',
  'MMZZZMMMMZMZZZZMZZZM'],
 ['DCBVPGCUTVWBVCXWNYQS',
  'VWXDJSWEFDCXDWBCLAIG',
  'RTVHTLTJLHFVHTDFXBRN',
  'HFDRFNFPNRTDRFVTBXHL',
  'NQTLDEQORLITLQFIJCAU',
  'LIFNVUIKHNQFNITQPWYE',
  'PLHJLBLFBJNHJLRNDVPX',
  'JNRPNXNTXPLRPNHLVDJB',
  'FKPTXQKYDTOPTKJOHESI',
  'TOJFBIOAVFKJFOPKRUGQ',
  'XRLBRPRVPBHLBRNHFTXJ',
  'BHNXHJHDJXRNXHLRTFBP',
  'BUNXHWUQJXENXULETSOC',
  'XELBRCEI

In [50]:
print(
    '     ' +
    ', '.join([str(decipher.a2i(a)).zfill(2) for a in decipher.actual_alphabet if decipher.a2i(a) % 2 == 0])
)

print(''.join(['-' for _ in range(83)]))
print('      Q   Q   P   R   N   G   K   S   S   N   Y   P   V   T   T   M   Z   F   P   K')
d = []
for item in distances:
    current = []
    i = 1
    print('      N   O   R   T   H   E   A   S   T   B   E   R   L   I   N   C   L   O   C   K')
    for string in item:
        current.append([str(decipher.a2i(t)).zfill(2) for t in string])
        print('{} | {}'.format(
            str(i).zfill(2),
            ', '.join(map(str, current[len(current)-1]))
        ))
        i += 1
    d.append(current)
    print()

     02, 04, 06, 08, 10, 12, 14, 16, 18, 20, 22, 24, 26
-----------------------------------------------------------------------------------
      Q   Q   P   R   N   G   K   S   S   N   Y   P   V   T   T   M   Z   F   P   K
      N   O   R   T   H   E   A   S   T   B   E   R   L   I   N   C   L   O   C   K
01 | 17, 17, 16, 18, 14, 07, 11, 19, 19, 14, 25, 16, 22, 20, 20, 13, 26, 06, 16, 11
02 | 09, 09, 10, 08, 12, 19, 15, 07, 07, 12, 01, 10, 04, 06, 06, 13, 26, 20, 10, 15
03 | 18, 18, 20, 16, 24, 12, 04, 14, 14, 24, 02, 20, 08, 12, 12, 26, 26, 14, 20, 04
04 | 08, 08, 06, 10, 02, 14, 22, 12, 12, 02, 24, 06, 18, 14, 14, 26, 26, 12, 06, 22
05 | 01, 01, 04, 24, 10, 05, 19, 21, 21, 10, 03, 04, 12, 18, 18, 13, 26, 08, 04, 19
06 | 25, 25, 22, 02, 16, 21, 07, 05, 05, 16, 23, 22, 14, 08, 08, 13, 26, 18, 22, 07
07 | 16, 16, 12, 20, 04, 02, 18, 24, 24, 04, 22, 12, 10, 02, 02, 26, 26, 24, 12, 18
08 | 10, 10, 14, 06, 22, 24, 08, 02, 02, 22, 04, 14, 16, 24, 24, 26, 26, 02, 14, 08
09 | 19, 19, 24, 14,

It's not really clear from these tables exactly what is going on but there is clearly something. This seems to be a drawback of attempting an entirely programatic solution to the unknown.

In the interim, I've taken these tables out and manipulated them within a [spreadsheet](pykryptos/alpha_tables.xlsx) which has shown something quite interesting.

### Column sorting
First I coloured each column so I could see how the columns distributed.
![colour_grading](pykryptos/graded_cipher.png)

Next I sorted the columns into numerical order based on the value at the top of each column. This provided a set of 13 columns above which I wrote out the alphabet according to the columns it was associated with in the ciphertext. Although we don't have a complete alphabet in the known ciphertext, there was enough data available to complete the set.

I then applied the same principle to the left and bottom of the table which left us with ![graded_evens](pykryptos/graded_evens.png)

Next we want to do the same with the mixed values from the collection. I didn't grade these as I already had an idea of what I was looking for (and it's a tedious process).

What we're left with is an incomplete alphabet.

![incomplete_alphabet.png](pykryptos/incomplete_alphabet.png)

With this, we know the values across the top row are sequential rising from 2, the second row is falling from 24, column 1 is missing so we can infer that the first two characters of this column are 1 and 25.

The last row of this column is also just as easy as this is again, a binary system between 13 and 26 so the last row of column 1 is 13.

We can go on to comlete column 1 by looking at the how the values are incrementing across the rest of the row which leads us to:

![completed_alpha_table.png](pykryptos/completed_alpha_table.png)

You'll notice that on the right hand side, I've written out the index of the ciphertext. That's because there is something else interesting here.

Take that index and modulate each value by 26.

In [51]:
rows = []
current_col = 0
for i in range(1, 98):
    val = i % 26 if i % 26 != 0 else 26
    
    if val == 1 or val == 14:
        current_col += 1 
    
    try:
        rows[current_col - 1]
    except IndexError:
        rows.append([])
    
    rows[current_col - 1].append(str(val).zfill(2))

# make all columns identical lengths for rotation:
# If we don't do this, we lose information from the table
# - we just need to delete this information afterwards
for i in range(len(rows)-1):
    while len(rows[i+1]) != len(rows[i]):
        rows[i+1].append('A')
rotated = list(zip(*reversed(rows)))
_ = [print(row[::-1]) for row in rotated]

('01', '14', '01', '14', '01', '14', '01', '14')
('02', '15', '02', '15', '02', '15', '02', '15')
('03', '16', '03', '16', '03', '16', '03', '16')
('04', '17', '04', '17', '04', '17', '04', '17')
('05', '18', '05', '18', '05', '18', '05', '18')
('06', '19', '06', '19', '06', '19', '06', '19')
('07', '20', '07', '20', '07', '20', '07', 'A')
('08', '21', '08', '21', '08', '21', '08', 'A')
('09', '22', '09', '22', '09', '22', '09', 'A')
('10', '23', '10', '23', '10', '23', '10', 'A')
('11', '24', '11', '24', '11', '24', '11', 'A')
('12', '25', '12', '25', '12', '25', '12', 'A')
('13', '26', '13', '26', '13', '26', '13', 'A')


Again we come back to the Berlin clock and the principle of the seconds indicator at the top of the clock, only this time we've got additional information being displayed. So let's put that in place on the table.

![completed_with_mask.png](pykryptos/completed_with_mask.png)

In each table, find the intersection of the current cipher character. Where M is found, we create a square through K with the K intersection in the top left and the M intersection in the bottom right
Where Z is found, use V in the top left
Where K is found, bottom right is M
where V is found, top left is K

```
           CIPHER                LACUNA
1  QQPRNGKSSNYPVTTMZFPK = IIJHLSOGGLAJDFFMZTJO
2  DCBVPGCUTVWBVCXWNYQS = VWXDJSWEFDCXDWBCLAIG
3  UTRNDNNNMJVRRWRJNEGD = EFHLVLLLMPDHHCHPLUSV
```

Mask

Find all positions a for character x in CIPHER[1] and all positions b for character y in LACUNA[1]

If the number found is in the mixed table, then the number belongs in CIPHER[2]. If it is in the evens table, the number found belongs in CIPHER[3]
If the number found is from LACUNA[1], subtract is first from Z to find CIPHER. 

> Note:
> Here, the inverse principle appears to apply. That is to say, if the answer is in the mixed table, it belongs in LACUNA[3]

If the number is in CIPHER[1], add this to `x % 26` to find M2
If the number is in M2, calculate the distance required to get from a to x, this is 

For example:

If the character from LACUNA[1] is S and the position found is N, to get from S to N we use `((Z - S) + N)`

There are a number of positions where the x and/or y are also found in the grid.

If a one of x or y is found, invert the cipher location
the presence of both x and y inverts the inversion?

In [52]:
evens_table = [
    [2,4,6,8,10,12,14,16,18,20,22,24,26],
    [24,22,20,18,16,14,12,10,8,6,4,2,26],
    [22,18,14,10,6,2,24,20,16,12,8,4,26],
    [4,8,12,16,20,24,2,6,10,14,18,22,26],
    [20,14,8,2,22,16,10,4,24,18,12,6,26],
    [6,12,18,24,4,10,16,22,2,8,14,20,26],
    [8,16,24,6,14,22,4,12,20,2,10,18,26],
    [18,10,2,20,12,4,22,14,6,24,16,8,26],
    [16,6,22,12,2,18,8,24,14,4,20,10,26],
    [10,20,4,14,24,8,18,2,12,22,6,16,26],
    [12,24,10,22,8,20,6,18,4,16,2,14,26],
    [14,2,16,4,18,6,20,8,22,10,24,12,26],
    [26,26,26,26,26,26,26,26,26,26,26,26,26],
]

mixed_table = [
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26],
    [25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,26],
    [23,20,17,14,11,8,5,2,25,22,19,16,13,10,7,4,1,24,21,18,15,12,9,6,3,26],
    [3,6,9,12,15,18,21,24,1,4,7,10,13,16,19,22,25,2,5,8,11,14,17,20,23,26],
    [21,16,11,6,1,22,17,12,7,2,23,18,13,8,3,24,19,14,9,4,25,20,15,10,5,26],
    [5,10,15,20,25,4,9,14,19,24,3,8,13,18,23,2,7,12,17,22,1,6,11,16,21,26],
    [7,14,21,2,9,16,23,4,11,18,25,6,13,20,1,8,15,22,3,10,17,24,5,12,19,26],
    [19,12,5,24,17,10,3,22,15,8,1,20,13,6,26,18,11,4,23,16,9,2,21,14,7,26],
    [17,8,25,16,7,24,15,6,23,14,5,22,13,4,21,12,3,20,11,2,19,10,1,18,9,26],
    [9,18,1,10,19,2,11,20,3,12,21,4,13,22,5,14,23,6,15,24,7,16,25,8,17,26],
    [11,22,7,18,3,14,25,10,21,6,17,2,13,24,9,20,5,16,1,12,23,8,19,4,15,26],
    [15,4,19,8,23,12,1,16,5,20,9,24,13,2,17,6,21,10,25,14,3,18,7,22,11,26],
    [13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26,13,26],
]

tables = {
    'evens': pd.DataFrame(evens_table),
    'mixed': pd.DataFrame(mixed_table)
}

In [53]:
tables['mixed'].loc[3, 9]

4

In [54]:
keys = {
    'left': [
        ('A', 'N'), ('O', 'B'), ('C', 'P'), ('Q', 'D'), ('E', 'R'),
        ('S', 'F'), ('G', 'T'), ('U', 'H'), ('I', 'V'), ('W', 'J'),
        ('K', 'X'), ('Y', 'L'), ('M', 'Z'),
    ],
    'right': [
        ('Y', 'L'), ('A', 'N'), ('O', 'B'), ('K', 'X'), ('C', 'P'),
        ('W', 'J'), ('I', 'V'), ('Q', 'D'), ('E', 'R'), ('U', 'H'),
        ('G', 'T'), ('S', 'F'), ('M', 'Z'),
    ],
    'evens': {
        'top': [
            ('S', 'F'), ('Y', 'L'), ('E', 'R'), ('K', 'X'), ('Q', 'D'), 
            ('W', 'J'), ('C', 'P'), ('I', 'V'), ('O', 'B'), ('U', 'H'),
            ('A', 'N'), ('G', 'T'), ('M', 'Z'),
        ],
        'bottom': [
            ('Y', 'L'), ('K', 'X'), ('W', 'J'), ('I', 'V'), ('U', 'H'),
            ('G', 'T'), ('S', 'F'), ('E', 'R'), ('Q', 'D'), ('C', 'P'),
            ('O', 'B'), ('A', 'N'), ('M', 'Z'),
        ],
    },
    'mixed': {
        'bottom': [
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
        ],
        
        'top': [
            'Z', 'Y', 'X', 'W', 'V', 'U', 'T', 'S', 'R', 'Q', 'P', 'O', 'N',
            'M', 'L', 'K', 'J', 'I', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A'
        ],
    }
}

In [55]:
import copy
class Highlighter(object):
    _df = None
    _applied = None
    _grid = None
    
    def __init__(self, df, grid):
        self._df = df
        self._grid = grid
        self._applied = self.apply_grid()
    
    def highlighty(self, df, color='yellow'):
        return 'background-color: {}'.format(color)

    def highlightr(self, df, color='#FF0000'):
        return 'background-color: {}; color: #FFFFFF'.format(color)

    def highlightg(self, df, color='#00FF00'):
        return 'background-color: {}'.format(color)

    def apply_grid(self):
        self._df.style.clear()
        return self._df.style.applymap(
            self.highlighty, subset=pd.IndexSlice[:, self._grid[0]]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[self._grid[1], :]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[:, self._grid[2]]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[self._grid[3], :]
        ).set_table_attributes(
            'style="font-size: 10px"'
        )
    
    def active_char(self, pos):
        return {
            'tl': self._df.loc[self._grid[1], self._grid[0]],
            'tr': self._df.loc[self._grid[1], self._grid[2]],
            'bl': self._df.loc[self._grid[3], self._grid[0]],
            'br': self._df.loc[self._grid[3], self._grid[2]],
        }[pos]

    def active(self, pos):
        {
            'tl': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[1], self._grid[0]]),
            'tr': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[1], self._grid[2]]),
            'bl': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[3], self._grid[0]]),
            'br': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[3], self._grid[2]]),
        }[pos](self._applied)
        return self

    def cipher(self, pos):
        {
            'tl': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[1], self._grid[0]]),
            'tr': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[1], self._grid[2]]),
            'bl': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[3], self._grid[0]]),
            'br': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[3], self._grid[2]]),
        }[pos](self._applied)
        return self
    
    def apply(self):
        return self._applied

In [56]:
class Square:
    tl = ''
    bl = ''
    tr = ''
    br = ''
    
    replace = {
        'M': 'K',
        'V': 'J',
        'Z': 'V',
        'K': 'V',
    }
    
    character = ''
    
    _grid = []
    
    _highlight = None
    lacuna_active = False
    cipher_active = False
    alt_active = False
    
    def __init__(self, character, table, tables, keys, use_alt):
        self.character = character
        character_index = decipher.a2i(character)
        
        alt_char = self.replace[character] if use_alt and character in self.replace.keys() else character
        self.alt_active = alt_char != character
        
        top    = [
            i for i in range(len(keys[table]['top'])) if alt_char in keys[table]['top'][i]
        ][0]
        
        right  = [
            i for i in range(len(keys['right'])) if alt_char in keys['right'][i]
        ][0]
        
        bottom = [
            i for i in range(len(keys[table]['bottom'])) if character in keys[table]['bottom'][i]
        ][0]
        
        left   = [
            i for i in range(len(keys['left'])) if character in keys['left'][i]
        ][0]
        
        table  = tables[table]
        
        self._grid = [
            top     if top < bottom else bottom,
            left    if left < right else right,
            bottom  if bottom > top else top,
            right   if right > left else left
        ]
        
        self.tl = table.loc[self._grid[1], self._grid[0]]
        self.tr = table.loc[self._grid[1], self._grid[2]]
        self.bl = table.loc[self._grid[3], self._grid[2]]
        self.br = table.loc[self._grid[3], self._grid[0]]
        self._highlight = Highlighter(table, self.gridref)
        self.markcipher(self.character)
    
    def get(self):
        return [self.tl, self.tr, self.br, self.bl]
    
    def markcipher(self, char, recurse=True):
        inverse = distancefrom(char, 'Z')
        char = decipher.a2i(char)
        pos = ''
        if char in self.get():
            pos = [
                'tl', 'tr', 'bl', 'br'
            ][self.get().index(char)]
            self._highlight.cipher(pos)
            
            if recurse:
                self.cipher_active = pos
            else:
                self.lacuna_active = pos
        if recurse:
            self.markcipher(inverse, False)
    
    def active(self, corner):
        self._highlight.active(corner)
        return self.active_char(corner)
    
    def active_char(self, corner):
        return self._highlight.active_char(corner)
        
    def contains(self, what):
        return decipher.a2i(what) in self.get()
            
    @property
    def gridref(self):
        return self._grid
    
    @property
    def apply(self):
        return self._highlight.apply()
s = Square('N', 'evens', tables, keys, False)
s.apply

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2,4,6,8,10,12,14,16,18,20,22,24,26
1,24,22,20,18,16,14,12,10,8,6,4,2,26
2,22,18,14,10,6,2,24,20,16,12,8,4,26
3,4,8,12,16,20,24,2,6,10,14,18,22,26
4,20,14,8,2,22,16,10,4,24,18,12,6,26
5,6,12,18,24,4,10,16,22,2,8,14,20,26
6,8,16,24,6,14,22,4,12,20,2,10,18,26
7,18,10,2,20,12,4,22,14,6,24,16,8,26
8,16,6,22,12,2,18,8,24,14,4,20,10,26
9,10,20,4,14,24,8,18,2,12,22,6,16,26


In [57]:
class Character:
    index  = 0
    cipher = {
        '+': None,
        '-': None,
    }

    character  = ''
    deciphered = ''

    table    = None
    xor_val  = None
    position = None
    cipher_active = False
    lacuna_active = False
    deciphered    = ''

    def __init__(self, character, index, tables, keys, use_alt=False):
        self.index       = index
        self.character   = character.upper()
        self.cipher['+'] = Square(self.character, 'evens', tables, keys, use_alt)
        self.cipher['-'] = Square(self.character, 'mixed', tables, keys, use_alt)
        self.xor_val = decipher.a2i(self.character) % 2 == 0
        self.deciphered = self.decipher()
        
    def __str__(self):
        return self.deciphered

    def __repr__(self):
        return str(self)

    def final(self):
        # D T X V D N C N N <- i need this
        # U T B V J S C G G <- i get this
        # 0 1 2 0 1 1 0 1 ? <- i need this
        # 0 1 2 0 0 1 1 0 0 <- i get this
        pos = 1 if self.table else 0
        pos += {
            'tl': 1 if self.lacuna_active and not self.cipher_active else 0,
            'bl': 0 if self.lacuna_active and not self.cipher_active else 1,
            'tr': 1 if self.lacuna_active and self.cipher_active else 0,
            'br': 0 if self.lacuna_active and not self.cipher_active else 1,
        }[self.position]
        
        pos = ((pos + 2) if self.xor_val else pos) % 4
        c = [
            lambda x, c: decipher.i2a( # D to U = (17 + D) % 26 
                (decipher.a2i(c) + decipher.a2i(x)) % 26
            ),
            lambda x, _: x,
            lambda x, c: decipher.i2a(
                (decipher.a2i(c) + decipher.a2i(distancefrom(x, 'Z'))) % 26
            ),
            lambda x, _: distancefrom(x, 'Z'), # should be mostly right
                
        ][pos](self.deciphered, self.character)
        return (pos, distancefrom(self.character, c))

    def decipher(self):
        self.cipher_active = self.cipher['+'].cipher_active \
            if self.cipher['+'].cipher_active else self.cipher['-'].cipher_active
        
        self.lacuna_active = self.cipher['+'].lacuna_active \
            if self.cipher['+'].lacuna_active else self.cipher['-'].lacuna_active
        self.position = 'tl'
        self.table = self.index % 2 != 0 and decipher.a2i(self.character) % 2 != 0
        
        # rules
        if self.lacuna_active:
            self.position = 'bl' if self.lacuna_active != 'bl' else 'br'
            self.table = not self.table

        if self.cipher_active and not self.lacuna_active:
            self.position = 'tr' if self.cipher_active != 'tr' else 'tl'
            self.table = not self.table

        if self.table:
            t = '+' if self.table else '-'
            u = '-' if self.table else '+'
            for i, c in zip(range(len(self.cipher[t].get())), self.cipher[t].get()):
                if (26 - c) in self.cipher[u].get():
                    self.position = [
                        'tl', 'tr', 'bl', 'br'
                    ][i]
                    break

            # if we have the same number in both tables
            # and shared number is in same position, invert the tables
            number = self.cipher[t].active_char(self.position)
            alternate = self.cipher[u].active_char(self.position)
            
            if number != alternate and alternate in self.cipher[t].get():
                self.table = not self.table
        
        # Now set table to the table key polarity
        table = '+' if self.table else '-'
        
        if self.cipher[table].alt_active:
            self.position = {
                'tl': 'bl',
                'tr': 'br',
                'bl': 'tl',
                'br': 'tr',
            }[self.position]
        number = self.cipher[table].active(self.position)
        return decipher.i2a(number)

In [58]:
class Cipher(object):
    cipher   = None
    lacuna   = None
    alphabet = None
    
    def __init__(self, ciphertext):
        self.ciphertext = ciphertext.upper()
        self.lacunatext = ''.join(
            [distancefrom(c, 'Z') for c in self.ciphertext]
        )
        
        self.alphabet = {
            'A': False, 'B': False, 'C': False, 'D': False, 'E': False, 'F': False,
            'G': False, 'H': False, 'I': False, 'J': False, 'K': False, 'L': False,
            'M': False, 'N': False, 'O': False, 'P': False, 'Q': False, 'R': False,
            'S': False, 'T': False, 'U': False, 'V': False, 'W': False, 'X': False,
            'Y': False, 'Z': False,
        }
        self.cipher = []
        self.lacuna = []
        for i, c, l in zip(range(1, self.length + 1), self.ciphertext, self.lacunatext):
            self.cipher.append(
                Character(c, i, tables, keys, self.alphabet[c])
            )

            self.lacuna.append(
                Character(l, i, tables, keys, self.alphabet[c])
            )
            
            self.alphabet[c] = not self.alphabet[c]

    @property
    def length(self):
        return len(self.ciphertext)

cipher = Cipher(ciphertext)
print(''.join([str(c) for c in cipher.cipher][25:34]))
print(''.join([str(c.final()) for c in cipher.cipher][25:34]))

DTXVDNCNN
(0, 'N')(1, 'O')(2, 'R')(3, 'D')(0, 'T')(2, 'Z')(1, 'L')(0, 'Z')(0, 'Z')


In [59]:
print(''.join([str(c.final()[1]) for c in cipher.cipher]))

FNGDGUHUZFGPNZFPKZNNKZPDQNORDTZLZZFXKXOZHNZZMWZLKZXHBPGDKZKWTZNTZRGXXZZZRWKODLLHXHYDKOGZFZYZMLYZD


In [60]:
cipher.cipher[25].cipher['-'].apply

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
1,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,26
2,23,20,17,14,11,8,5,2,25,22,19,16,13,10,7,4,1,24,21,18,15,12,9,6,3,26
3,3,6,9,12,15,18,21,24,1,4,7,10,13,16,19,22,25,2,5,8,11,14,17,20,23,26
4,21,16,11,6,1,22,17,12,7,2,23,18,13,8,3,24,19,14,9,4,25,20,15,10,5,26
5,5,10,15,20,25,4,9,14,19,24,3,8,13,18,23,2,7,12,17,22,1,6,11,16,21,26
6,7,14,21,2,9,16,23,4,11,18,25,6,13,20,1,8,15,22,3,10,17,24,5,12,19,26
7,19,12,5,24,17,10,3,22,15,8,1,20,13,6,26,18,11,4,23,16,9,2,21,14,7,26
8,17,8,25,16,7,24,15,6,23,14,5,22,13,4,21,12,3,20,11,2,19,10,1,18,9,26
9,9,18,1,10,19,2,11,20,3,12,21,4,13,22,5,14,23,6,15,24,7,16,25,8,17,26


In [61]:
decipher.actual_alphabet

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']